# Imports

All the libraries used in the notebook

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
from collections import defaultdict
import math
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
%matplotlib notebook
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource
from bokeh.palettes import Category20
output_notebook()
pd.options.display.float_format = '{:.5f}'.format

Loading BokehJS ...

# Technical Functions

The below functions are the fundemental aspects of this project, they are used to calculate the stabilities for each pertubation.

In [2]:
def init_graph(G,node_adj_frame):
    G.add_nodes_from([i for i in range(len(node_adj_frame))])
    labels = {}
    labels = node_adj_frame.columns
    for i in range(len(node_adj_frame)):
        snode = node_adj_frame[labels[0]][i]-1
        temp = node_adj_frame[labels[2]][i]
        if ',' in str(temp):
            sedge_arr = temp.split(',')
            for j in range(0, len(sedge_arr)):
                k = int(sedge_arr[j])
                G.add_edge(snode, k-1)
        elif np.isnan(temp):
            print("ERROR: Not found in the adjacency excel sheet")
        else:
            G.add_edge(snode, int(temp)-1)
    return

In [3]:
def init_graph_attr(G, AdjFile, df, columns):
    node_adj_frame = pd.read_excel(AdjFile)
    node_list = node_adj_frame["District_Name"].tolist()
    nodeAttr = {}
    init_graph(G, node_adj_frame)
    
    capability_vector = list(zip(*(df[col] for col in columns)))
    node_attri_dict = dict(zip(df["District"], capability_vector))
    node_attri_dict = dict((k, v) for k, v in node_attri_dict.items())

    for i in range(len(node_adj_frame)):
        temp = {}
        temp["capabilityvector"] = node_attri_dict[node_list[i]]
        temp["nodeStress"] = 0
        temp["nodeStability"] = 0
        temp["name"] = node_list[i]
        nodeAttr[i] = temp

    nx.set_node_attributes(G, nodeAttr)

In [4]:
def addList(l1,l2):
    for i in range(len(l1)):
        l1[i] = l1[i] + l2[i]
    return l1

def divList(l1,k):
    for i in range(len(l1)):
        l1[i] = l1[i]/k
    return l1
def l2_normalization(l1,l2):
    k = 0
    for i in range(len(l1)):
        k+= (l1[i] - l2[i])**2
    return math.sqrt(k)

In [5]:
def get_node_stress(G,dim):
    stress_dict = {}
    for n in G.nodes():
        centroid = [0]*dim
        neighList = list(G.neighbors(n))
        for nei in neighList:
            try:
                centroid = addList(centroid,list(G.nodes[nei]["capabilityvector"]))
            except(KeyError):
                pass
        try:
            G.nodes[n]["nodeStress"] = l2_normalization(divList(centroid,len(neighList)),list(G.nodes[n]["capabilityvector"]))
        except(KeyError):
            pass
        try:
            stress_dict[G.nodes[n]["name"]]=G.nodes[n]["nodeStress"]
        except(KeyError):
            pass
    return stress_dict

In [6]:
def get_node_stability(G,dim):
    stability_dict = {}
    for n in G.nodes():
        centroid = [0]*dim
        neighList = list(G.neighbors(n))
        for nei in neighList:
            try:
                centroid = addList(centroid,list(G.nodes[nei]["capabilityvector"]))
            except(KeyError):
                pass
        try:
            G.nodes[n]["nodeStability"] = 1 - l2_normalization(divList(centroid,len(neighList)),list(G.nodes[n]["capabilityvector"]))
        except(KeyError):
            pass
        try:
            stability_dict[G.nodes[n]["name"]]=G.nodes[n]["nodeStability"]
        except(KeyError):
            pass
    return stability_dict

In [7]:
G = nx.Graph()
df = pd.read_csv('KAG 2016-17/Agriculture/Agriculture_KAG_2016_17.csv')
adjacency_file = 'Karnataka_District_Adjacency_File - Copy.xlsx'

# existing_data = pd.DataFrame(df['District'])

existing_data = pd.read_csv('result.csv')
# adjacency_file = "KAG 2016-17/Karnataka_District_Adjacency_File.xlsx"

In [8]:
df.columns

Index(['District', 'NetAreaIrrigated_Canals_Length_171',
       'NetAreaIrrigated_Canals_GrossIrrigatedArea_172',
       'NetAreaIrrigated_Canals_NetAreaIrrigated_173',
       'NetAreaIrrigated_Tanks_No_174',
       'NetAreaIrrigated_Tanks_GrossIrrigatedArea_175',
       'NetAreaIrrigated_Tanks_NetAreaIrrigated_176',
       'NetAreaIrrigated_Wells_No_177',
       'NetAreaIrrigated_Wells_GrossIrrigatedArea_178',
       'NetAreaIrrigated_Wells_NetAreaIrrigated_179',
       ...
       'SowingSeedsDistributed_Safflower_323',
       'SowingSeedsDistributed_Groundnut_328',
       'SowingSeedsDistributed_Sunflower_329',
       'SowingSeedsDistributed_Soyabean_330', 'TotalFoodGrains_240',
       'TotalGourdVarietyVegetables_276', 'RRB_AgricultureLoan_448',
       'DCCBank_AgricultureLoan_466', 'KSCARD_PLDBank_AgricultureLoan_472',
       'TotalAgricultureLoan'],
      dtype='object', length=302)

Removing the {_id} from the column names

In [9]:
import re
def remove_pattern(col_name):
    return re.sub(r'_[0-9]{3}$', '', col_name)

df.columns = map(remove_pattern, df.columns)

In [10]:
df = df[['District','TotalNPK', 'Rice_Production','Jowar_Production','Maize_Production','Jowar_Yield','TotalCerealsandMinorMillets_Production','TotalOilSeeds_Production','TotalFoodGrains','TotalGourdVarietyVegetables']].copy()

In [11]:
# df['Total_Production']

In [12]:
df

,District,TotalNPK,Rice_Production,Jowar_Production,Maize_Production,Jowar_Yield,TotalCerealsandMinorMillets_Production,TotalOilSeeds_Production,TotalFoodGrains,TotalGourdVarietyVegetables
0,BENGALURU,23310,3376,851,2879,1539,69153,162,72752,11088
1,BENGALURU(R),19259,2354,0,42583,0,125232,479,131795,12614
2,RAMANAGARA,7472,12128,0,10450,0,159008,6414,175873,10071
3,CHITRADURGA,35884,4566,13301,271975,984,388849,78257,421955,6113
4,DAVANAGERE,99644,489505,23072,593544,2167,1126753,21327,1141132,11822
5,KOLAR,22190,864,0,1759,0,69089,5994,87015,14692
6,CHIKKABALLAPURA,28357,2698,0,105491,0,165723,7341,171962,19918
7,SHIVAMOGGA,56609,384974,756,226532,2242,613331,2146,613998,238
8,TUMAKURU,38148,26042,2652,66868,1346,431562,42331,450563,4382
9,CHIKKAMAGALURU,67863,110533,9080,61353,897,235695,8612,253087,13083


Adding the District Vijayanagara into the dataframe

In [13]:
vijayanagara_values = df[df['District'] == 'BALLARI'].iloc[0].copy()
vijayanagara_values['District'] = 'Vijayanagara'
vijayanagara_values[['TotalNPK', 'Rice_Production', 'Jowar_Production', 'Maize_Production', 'Jowar_Yield','TotalCerealsandMinorMillets_Production','TotalOilSeeds_Production','TotalFoodGrains','TotalGourdVarietyVegetables']] = (vijayanagara_values[['TotalNPK', 'Rice_Production', 'Jowar_Production', 'Maize_Production', 'Jowar_Yield','TotalCerealsandMinorMillets_Production','TotalOilSeeds_Production','TotalFoodGrains','TotalGourdVarietyVegetables']].astype(float) / 2).round().astype(int)
df = pd.concat([df, pd.DataFrame([vijayanagara_values], columns=df.columns)], ignore_index=True)

df.loc[df['District'] == 'BALLARI', ['TotalNPK', 'Rice_Production', 'Jowar_Production', 'Maize_Production', 'Jowar_Yield','TotalCerealsandMinorMillets_Production','TotalOilSeeds_Production','TotalFoodGrains','TotalGourdVarietyVegetables']] /= 2
df.loc[df['District'] == 'BALLARI', ['TotalNPK', 'Rice_Production', 'Jowar_Production', 'Maize_Production', 'Jowar_Yield','TotalCerealsandMinorMillets_Production','TotalOilSeeds_Production','TotalFoodGrains','TotalGourdVarietyVegetables']] = df.loc[df['District'] == 'BALLARI', ['TotalNPK', 'Rice_Production', 'Jowar_Production', 'Maize_Production', 'Jowar_Yield','TotalCerealsandMinorMillets_Production','TotalOilSeeds_Production','TotalFoodGrains','TotalGourdVarietyVegetables']].applymap(lambda x: x + 0.5 if x % 1 != 0 else x)

Taking the ratio of TotalNPK and Total Production to get the amount of NPK for each crop (estimated proportion)

In [14]:
df['Total_Production'] = df['TotalCerealsandMinorMillets_Production'] + df['TotalOilSeeds_Production'] + df['TotalFoodGrains'] + df['TotalGourdVarietyVegetables']
df['FCR'] = df['TotalNPK']/df['Total_Production']

In [15]:
df

,District,TotalNPK,Rice_Production,Jowar_Production,Maize_Production,Jowar_Yield,TotalCerealsandMinorMillets_Production,TotalOilSeeds_Production,TotalFoodGrains,TotalGourdVarietyVegetables,Total_Production,FCR
0,BENGALURU,23310.00000,3376.00000,851,2879,1539.00000,69153.00000,162.00000,72752.00000,11088.00000,153155.00000,0.15220
1,BENGALURU(R),19259.00000,2354.00000,0,42583,0.00000,125232.00000,479.00000,131795.00000,12614.00000,270120.00000,0.07130
2,RAMANAGARA,7472.00000,12128.00000,0,10450,0.00000,159008.00000,6414.00000,175873.00000,10071.00000,351366.00000,0.02127
3,CHITRADURGA,35884.00000,4566.00000,13301,271975,984.00000,388849.00000,78257.00000,421955.00000,6113.00000,895174.00000,0.04009
4,DAVANAGERE,99644.00000,489505.00000,23072,593544,2167.00000,1126753.00000,21327.00000,1141132.00000,11822.00000,2301034.00000,0.04330
5,KOLAR,22190.00000,864.00000,0,1759,0.00000,69089.00000,5994.00000,87015.00000,14692.00000,176790.00000,0.12552
6,CHIKKABALLAPURA,28357.00000,2698.00000,0,105491,0.00000,165723.00000,7341.00000,171962.00000,19918.00000,364944.00000,0.07770
7,SHIVAMOGGA,56609.00000,384974.00000,756,226532,2242.00000,613331.00000,2146.00000,613998.00000,238.00000,1229713.00000,0.04603
8,TUMAKURU,38148.00000,26042.00000,2652,66868,1346.00000,431562.00000,42331.00000,450563.00000,4382.00000,928838.00000,0.04107
9,CHIKKAMAGALURU,67863.00000,110533.00000,9080,61353,897.00000,235695.00000,8612.00000,253087.00000,13083.00000,510477.00000,0.13294


In [16]:
df['FCR'] = df['TotalNPK']/df['Total_Production']

In [17]:
df

,District,TotalNPK,Rice_Production,Jowar_Production,Maize_Production,Jowar_Yield,TotalCerealsandMinorMillets_Production,TotalOilSeeds_Production,TotalFoodGrains,TotalGourdVarietyVegetables,Total_Production,FCR
0,BENGALURU,23310.00000,3376.00000,851,2879,1539.00000,69153.00000,162.00000,72752.00000,11088.00000,153155.00000,0.15220
1,BENGALURU(R),19259.00000,2354.00000,0,42583,0.00000,125232.00000,479.00000,131795.00000,12614.00000,270120.00000,0.07130
2,RAMANAGARA,7472.00000,12128.00000,0,10450,0.00000,159008.00000,6414.00000,175873.00000,10071.00000,351366.00000,0.02127
3,CHITRADURGA,35884.00000,4566.00000,13301,271975,984.00000,388849.00000,78257.00000,421955.00000,6113.00000,895174.00000,0.04009
4,DAVANAGERE,99644.00000,489505.00000,23072,593544,2167.00000,1126753.00000,21327.00000,1141132.00000,11822.00000,2301034.00000,0.04330
5,KOLAR,22190.00000,864.00000,0,1759,0.00000,69089.00000,5994.00000,87015.00000,14692.00000,176790.00000,0.12552
6,CHIKKABALLAPURA,28357.00000,2698.00000,0,105491,0.00000,165723.00000,7341.00000,171962.00000,19918.00000,364944.00000,0.07770
7,SHIVAMOGGA,56609.00000,384974.00000,756,226532,2242.00000,613331.00000,2146.00000,613998.00000,238.00000,1229713.00000,0.04603
8,TUMAKURU,38148.00000,26042.00000,2652,66868,1346.00000,431562.00000,42331.00000,450563.00000,4382.00000,928838.00000,0.04107
9,CHIKKAMAGALURU,67863.00000,110533.00000,9080,61353,897.00000,235695.00000,8612.00000,253087.00000,13083.00000,510477.00000,0.13294


NPK/TP * Crop

We are finding the proportional allocation based on the crop production to total production

In [18]:
df['NPK_Rice'] = df['TotalNPK']*df['Rice_Production']/df['Total_Production']
df['NPK_Maize'] = df['TotalNPK']*df['Maize_Production']/df['Total_Production']

In [19]:
df

,District,TotalNPK,Rice_Production,Jowar_Production,Maize_Production,Jowar_Yield,TotalCerealsandMinorMillets_Production,TotalOilSeeds_Production,TotalFoodGrains,TotalGourdVarietyVegetables,Total_Production,FCR,NPK_Rice,NPK_Maize
0,BENGALURU,23310.00000,3376.00000,851,2879,1539.00000,69153.00000,162.00000,72752.00000,11088.00000,153155.00000,0.15220,513.82299,438.18021
1,BENGALURU(R),19259.00000,2354.00000,0,42583,0.00000,125232.00000,479.00000,131795.00000,12614.00000,270120.00000,0.07130,167.83535,3036.08025
2,RAMANAGARA,7472.00000,12128.00000,0,10450,0.00000,159008.00000,6414.00000,175873.00000,10071.00000,351366.00000,0.02127,257.90889,222.22526
3,CHITRADURGA,35884.00000,4566.00000,13301,271975,984.00000,388849.00000,78257.00000,421955.00000,6113.00000,895174.00000,0.04009,183.03296,10902.40657
4,DAVANAGERE,99644.00000,489505.00000,23072,593544,2167.00000,1126753.00000,21327.00000,1141132.00000,11822.00000,2301034.00000,0.04330,21197.52955,25702.83548
5,KOLAR,22190.00000,864.00000,0,1759,0.00000,69089.00000,5994.00000,87015.00000,14692.00000,176790.00000,0.12552,108.44595,220.78291
6,CHIKKABALLAPURA,28357.00000,2698.00000,0,105491,0.00000,165723.00000,7341.00000,171962.00000,19918.00000,364944.00000,0.07770,209.64089,8196.89675
7,SHIVAMOGGA,56609.00000,384974.00000,756,226532,2242.00000,613331.00000,2146.00000,613998.00000,238.00000,1229713.00000,0.04603,17722.01576,10428.24626
8,TUMAKURU,38148.00000,26042.00000,2652,66868,1346.00000,431562.00000,42331.00000,450563.00000,4382.00000,928838.00000,0.04107,1069.56242,2746.31363
9,CHIKKAMAGALURU,67863.00000,110533.00000,9080,61353,897.00000,235695.00000,8612.00000,253087.00000,13083.00000,510477.00000,0.13294,14694.29765,8156.29037


In [20]:
df_new = df[['Total_Production', 'NPK_Rice', 'NPK_Maize']]

In [21]:
df_new.to_excel('SGD2_FCR.xlsx')

Checking how the new data with NPK proportioned to each crop is different from the original

In [22]:
base_column = "TotalNPK"
capability_vector = "Rice_Production"
X = sm.add_constant(df[base_column])
y = df[capability_vector]
model = sm.OLS(y, X).fit()
m, c = model.params[base_column], model.params['const']
print(m,c)

2.1764894064966205 17136.471837471167


In [23]:
base_column = "NPK_Rice"
capability_vector = "Rice_Production"
X = sm.add_constant(df[base_column])
y = df[capability_vector]
model = sm.OLS(y, X).fit()
m, c = model.params[base_column], model.params['const']
print(m,c)

11.982189439376699 21969.740534991743


In [24]:
base_column = "TotalNPK"
capability_vector = "Maize_Production"
X = sm.add_constant(df[base_column])
y = df[capability_vector]
model = sm.OLS(y, X).fit()
m, c = model.params[base_column], model.params['const']
print(m,c)

2.731783584846858 -8203.94439643982


In [25]:
base_column = "NPK_Maize"
capability_vector = "Maize_Production"
X = sm.add_constant(df[base_column])
y = df[capability_vector]
model = sm.OLS(y, X).fit()
m, c = model.params[base_column], model.params['const']
print(m,c)

14.592486866377042 16260.762174034237


In [26]:
def calculate_and_return_initial_stability(df, columns, G, adjacency_file = adjacency_file):
    dim = len(columns)
    col_to_pass = []

    for column in columns:
        column_values = df[column].values.reshape(-1, 1)
        scaler = MinMaxScaler()
        normalized_column_values = scaler.fit_transform(column_values)
        df[f"Normalized_{column}"] = normalized_column_values
        col_to_pass.append(f"Normalized_{column}")

    init_graph_attr(G, adjacency_file, df, col_to_pass)
    initial_stability = get_node_stability(G, dim)
    df["Initial Stability"] = df["District"].map(initial_stability)

    return df["Initial Stability"]

In [27]:
df

,District,TotalNPK,Rice_Production,Jowar_Production,Maize_Production,Jowar_Yield,TotalCerealsandMinorMillets_Production,TotalOilSeeds_Production,TotalFoodGrains,TotalGourdVarietyVegetables,Total_Production,FCR,NPK_Rice,NPK_Maize
0,BENGALURU,23310.00000,3376.00000,851,2879,1539.00000,69153.00000,162.00000,72752.00000,11088.00000,153155.00000,0.15220,513.82299,438.18021
1,BENGALURU(R),19259.00000,2354.00000,0,42583,0.00000,125232.00000,479.00000,131795.00000,12614.00000,270120.00000,0.07130,167.83535,3036.08025
2,RAMANAGARA,7472.00000,12128.00000,0,10450,0.00000,159008.00000,6414.00000,175873.00000,10071.00000,351366.00000,0.02127,257.90889,222.22526
3,CHITRADURGA,35884.00000,4566.00000,13301,271975,984.00000,388849.00000,78257.00000,421955.00000,6113.00000,895174.00000,0.04009,183.03296,10902.40657
4,DAVANAGERE,99644.00000,489505.00000,23072,593544,2167.00000,1126753.00000,21327.00000,1141132.00000,11822.00000,2301034.00000,0.04330,21197.52955,25702.83548
5,KOLAR,22190.00000,864.00000,0,1759,0.00000,69089.00000,5994.00000,87015.00000,14692.00000,176790.00000,0.12552,108.44595,220.78291
6,CHIKKABALLAPURA,28357.00000,2698.00000,0,105491,0.00000,165723.00000,7341.00000,171962.00000,19918.00000,364944.00000,0.07770,209.64089,8196.89675
7,SHIVAMOGGA,56609.00000,384974.00000,756,226532,2242.00000,613331.00000,2146.00000,613998.00000,238.00000,1229713.00000,0.04603,17722.01576,10428.24626
8,TUMAKURU,38148.00000,26042.00000,2652,66868,1346.00000,431562.00000,42331.00000,450563.00000,4382.00000,928838.00000,0.04107,1069.56242,2746.31363
9,CHIKKAMAGALURU,67863.00000,110533.00000,9080,61353,897.00000,235695.00000,8612.00000,253087.00000,13083.00000,510477.00000,0.13294,14694.29765,8156.29037


Getting the Normalized Values for the production values for each crop for later usage

In [28]:
columns_to_normalize = ["Maize_Production","Rice_Production","Jowar_Production"]
dim = 1
for col in columns_to_normalize:
    column_values = df[col].values.reshape(-1, 1)
    scaler = MinMaxScaler()
    normalized_values = scaler.fit_transform(column_values)
    df[f"Normalized_{col}"] = normalized_values.flatten()
    init_graph_attr(G,adjacency_file , df,[f"Normalized_{col}"] )
    initial_stability = get_node_stability(G, dim)
    df[f"Initial Stability_{col}"] = df["District"].map(initial_stability)

In [29]:
df

,District,TotalNPK,Rice_Production,Jowar_Production,Maize_Production,Jowar_Yield,TotalCerealsandMinorMillets_Production,TotalOilSeeds_Production,TotalFoodGrains,TotalGourdVarietyVegetables,Total_Production,FCR,NPK_Rice,NPK_Maize,Normalized_Maize_Production,Initial Stability_Maize_Production,Normalized_Rice_Production,Initial Stability_Rice_Production,Normalized_Jowar_Production,Initial Stability_Jowar_Production
0,BENGALURU,23310.00000,3376.00000,851,2879,1539.00000,69153.00000,162.00000,72752.00000,11088.00000,153155.00000,0.15220,513.82299,438.18021,0.00485,0.97408,0.00669,0.99647,0.00327,0.99673
1,BENGALURU(R),19259.00000,2354.00000,0,42583,0.00000,125232.00000,479.00000,131795.00000,12614.00000,270120.00000,0.07130,167.83535,3036.08025,0.07174,0.99142,0.00461,0.98646,0.00000,0.99731
2,RAMANAGARA,7472.00000,12128.00000,0,10450,0.00000,159008.00000,6414.00000,175873.00000,10071.00000,351366.00000,0.02127,257.90889,222.22526,0.01761,0.92339,0.02446,0.89293,0.00000,0.99260
3,CHITRADURGA,35884.00000,4566.00000,13301,271975,984.00000,388849.00000,78257.00000,421955.00000,6113.00000,895174.00000,0.04009,183.03296,10902.40657,0.45822,0.85510,0.00910,0.58742,0.05106,0.97219
4,DAVANAGERE,99644.00000,489505.00000,23072,593544,2167.00000,1126753.00000,21327.00000,1141132.00000,11822.00000,2301034.00000,0.04330,21197.52955,25702.83548,1.00000,0.33175,0.99424,0.35694,0.08857,0.99307
5,KOLAR,22190.00000,864.00000,0,1759,0.00000,69089.00000,5994.00000,87015.00000,14692.00000,176790.00000,0.12552,108.44595,220.78291,0.00296,0.91819,0.00158,0.99605,0.00000,0.99891
6,CHIKKABALLAPURA,28357.00000,2698.00000,0,105491,0.00000,165723.00000,7341.00000,171962.00000,19918.00000,364944.00000,0.07770,209.64089,8196.89675,0.17773,0.88472,0.00531,0.98567,0.00000,0.99661
7,SHIVAMOGGA,56609.00000,384974.00000,756,226532,2242.00000,613331.00000,2146.00000,613998.00000,238.00000,1229713.00000,0.04603,17722.01576,10428.24626,0.38166,0.98650,0.78189,0.63098,0.00290,0.95016
8,TUMAKURU,38148.00000,26042.00000,2652,66868,1346.00000,431562.00000,42331.00000,450563.00000,4382.00000,928838.00000,0.04107,1069.56242,2746.31363,0.11266,0.90924,0.05273,0.90126,0.01018,0.99672
9,CHIKKAMAGALURU,67863.00000,110533.00000,9080,61353,897.00000,235695.00000,8612.00000,253087.00000,13083.00000,510477.00000,0.13294,14694.29765,8156.29037,0.10337,0.74403,0.22437,0.84160,0.03486,0.98750


In [30]:
existing_data = pd.DataFrame(df['District'])

In [31]:
dim = 1

The Code to get the stability of a column(s)

In [32]:
def calculate_and_map_stability(G, existing_data, adjacency_file, columns_to_pass, inter, dim):
    # Initialize the graph attributes
    init_graph_attr(G, adjacency_file, existing_data, columns_to_pass)

    # Calculate node stress for the given change percentage
    NPK_stability = get_node_stability(G, dim)

    # Create a new column in the result DataFrame
    stability_column_name = f"New Stability(NPK {'+' if inter >= 0 else '-'} {abs(inter)}%)"
    stability_column = existing_data["District"].map(NPK_stability)

    return stability_column, stability_column_name

In [33]:
# init_graph_attr(G,adjacency_file , new_data_20,[f"Normalized {capability_vector} ({base_column} {'+' if intervention >= 0 else '-'}{abs(intervention)}%)"] )
# stability_20 = get_node_stability(G, dim)
# df[f"Stability_20_{col}"] = df["District"].map(stability_20)

In [34]:
# new_data_20

In [35]:

# initial_stability_column = calculate_and_return_initial_stability(df, capability_vector, G)

The Code to get the stability of a column(s)

In [36]:
def calculate_and_map_stress(G, existing_data, adjacency_file, columns_to_pass, inter, dim):
    # Initialize the graph attributes
    init_graph_attr(G, adjacency_file, existing_data, columns_to_pass)

    # Calculate node stress for the given change percentage
    NPK_stress = get_node_stress(G, dim)

    # Create a new column in the result DataFrame
    stability_column_name = f"New Stability(NPK {'+' if inter >= 0 else '-'} {abs(inter)}%)"
    stability_column = existing_data["District"].map(NPK_stress)

    return stability_column, stability_column_name

## 1D Capability Vector

The Code to get the new production values and the stability of the perturbation for a single Capability vector and the visualize it.

In [37]:
df

,District,TotalNPK,Rice_Production,Jowar_Production,Maize_Production,Jowar_Yield,TotalCerealsandMinorMillets_Production,TotalOilSeeds_Production,TotalFoodGrains,TotalGourdVarietyVegetables,Total_Production,FCR,NPK_Rice,NPK_Maize,Normalized_Maize_Production,Initial Stability_Maize_Production,Normalized_Rice_Production,Initial Stability_Rice_Production,Normalized_Jowar_Production,Initial Stability_Jowar_Production
0,BENGALURU,23310.00000,3376.00000,851,2879,1539.00000,69153.00000,162.00000,72752.00000,11088.00000,153155.00000,0.15220,513.82299,438.18021,0.00485,0.97408,0.00669,0.99647,0.00327,0.99673
1,BENGALURU(R),19259.00000,2354.00000,0,42583,0.00000,125232.00000,479.00000,131795.00000,12614.00000,270120.00000,0.07130,167.83535,3036.08025,0.07174,0.99142,0.00461,0.98646,0.00000,0.99731
2,RAMANAGARA,7472.00000,12128.00000,0,10450,0.00000,159008.00000,6414.00000,175873.00000,10071.00000,351366.00000,0.02127,257.90889,222.22526,0.01761,0.92339,0.02446,0.89293,0.00000,0.99260
3,CHITRADURGA,35884.00000,4566.00000,13301,271975,984.00000,388849.00000,78257.00000,421955.00000,6113.00000,895174.00000,0.04009,183.03296,10902.40657,0.45822,0.85510,0.00910,0.58742,0.05106,0.97219
4,DAVANAGERE,99644.00000,489505.00000,23072,593544,2167.00000,1126753.00000,21327.00000,1141132.00000,11822.00000,2301034.00000,0.04330,21197.52955,25702.83548,1.00000,0.33175,0.99424,0.35694,0.08857,0.99307
5,KOLAR,22190.00000,864.00000,0,1759,0.00000,69089.00000,5994.00000,87015.00000,14692.00000,176790.00000,0.12552,108.44595,220.78291,0.00296,0.91819,0.00158,0.99605,0.00000,0.99891
6,CHIKKABALLAPURA,28357.00000,2698.00000,0,105491,0.00000,165723.00000,7341.00000,171962.00000,19918.00000,364944.00000,0.07770,209.64089,8196.89675,0.17773,0.88472,0.00531,0.98567,0.00000,0.99661
7,SHIVAMOGGA,56609.00000,384974.00000,756,226532,2242.00000,613331.00000,2146.00000,613998.00000,238.00000,1229713.00000,0.04603,17722.01576,10428.24626,0.38166,0.98650,0.78189,0.63098,0.00290,0.95016
8,TUMAKURU,38148.00000,26042.00000,2652,66868,1346.00000,431562.00000,42331.00000,450563.00000,4382.00000,928838.00000,0.04107,1069.56242,2746.31363,0.11266,0.90924,0.05273,0.90126,0.01018,0.99672
9,CHIKKAMAGALURU,67863.00000,110533.00000,9080,61353,897.00000,235695.00000,8612.00000,253087.00000,13083.00000,510477.00000,0.13294,14694.29765,8156.29037,0.10337,0.74403,0.22437,0.84160,0.03486,0.98750


In [38]:
from bokeh.models import Span, Label
from bokeh.plotting import figure, show, output_notebook
from bokeh.models.sources import ColumnDataSource
from bokeh.models.tools import HoverTool
import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler
import plotly.express as px

def calc_and_vis_impact_stability_1D(G, existing_data, adjacency_file, base_column, capability_vector, intervention, dim):
    filtered_df = df[(df[capability_vector] != 0) & (~df[capability_vector].isnull())]
    # df['temp'] = df[base_column]*df[capability_vector]
    X = sm.add_constant(df[base_column])
    y = df[capability_vector]
    model = sm.OLS(y, X).fit()
    
    m, c = model.params[base_column], model.params['const']
    print(m,c)
    base_column_increase = (1 + intervention/100) * df[base_column]
    base_column_decrease = (1 - intervention/100) * df[base_column]

    new_vector_increase = m * base_column_increase + c
    new_vector_decrease = m * base_column_decrease + c
    
    vector_plus = new_vector_increase - m * df[base_column] - c  
    vector_minus = new_vector_decrease - m * df[base_column] - c

    scaler = MinMaxScaler()
    normalized_change_increase_vector = scaler.fit_transform(vector_plus.values.reshape(-1, 1))
    normalized_change_decrease_vector = scaler.fit_transform(vector_minus.values.reshape(-1, 1))
    normalized_new_increase_vector = scaler.fit_transform(new_vector_increase.values.reshape(-1, 1))
    normalized_new_decrease_vector = scaler.fit_transform(new_vector_decrease.values.reshape(-1, 1))

    result_df = pd.DataFrame({
        f'{capability_vector} ({base_column} +{intervention}%)': new_vector_increase,
        f'{capability_vector} ({base_column} -{intervention}%)': new_vector_decrease,
        f'Normalized {capability_vector} ({base_column} +{intervention}%)': normalized_new_increase_vector.flatten(),
        f'Normalized {capability_vector} ({base_column} -{intervention}%)': normalized_new_decrease_vector.flatten(),
        f'Impact Score {capability_vector} ({base_column} +{intervention}%)': normalized_change_increase_vector.flatten(),
        f'Impact Score {capability_vector} ({base_column} -{intervention}%)': normalized_change_decrease_vector.flatten(),
    })
    
    # Merge Columns
    for column in result_df.columns:
        existing_data[column] = result_df[column]

    # Calculate Stability and add to DataFrame
    columns_to_pass = [f"Normalized {capability_vector} ({base_column} {'+' if intervention >= 0 else '-'}{abs(intervention)}%)"]
    stability_column, stability_column_name = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, columns_to_pass, intervention, dim)

    existing_data[stability_column_name] = stability_column

    # Calculate Stability for the opposite intervention and add to DataFrame
    opposite_intervention = -intervention
    columns_to_pass_opposite = [f"Normalized {capability_vector} ({base_column} {'+' if opposite_intervention >= 0 else '-'}{abs(opposite_intervention)}%)"]
    stability_column_opposite, stability_column_name_opposite = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, columns_to_pass_opposite, opposite_intervention, dim)

    existing_data[stability_column_name_opposite] = stability_column_opposite

    columns_to_pass = [f"Normalized {capability_vector} ({base_column} {'+' if intervention >= 0 else '-'}{abs(intervention)}%)"]
    stability_column, stability_column_name = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, columns_to_pass, intervention, dim)

    existing_data[stability_column_name] = stability_column

    # Calculate Stability for the opposite intervention and add to DataFrame
    opposite_intervention = -intervention
    columns_to_pass_opposite = [f"Normalized {capability_vector} ({base_column} {'+' if opposite_intervention >= 0 else '-'}{abs(opposite_intervention)}%)"]
    stability_column_opposite, stability_column_name_opposite = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, columns_to_pass_opposite, opposite_intervention, dim)

    existing_data[stability_column_name_opposite] = stability_column_opposite

    # Visualize with Plotly for +intervention
    fig_increase = px.scatter(existing_data,
                              x=stability_column_name,
                              y=f'Impact Score {capability_vector} ({base_column} +{intervention}%)',
                              title=f'Impact vs Stability ({capability_vector} - {base_column} +{intervention}%)',
                              labels={'x': f'Stability ({base_column} +{intervention}%)', 'y': f'Impact ({capability_vector})'},
                              size_max=8,
                              width=800,
                              height=500)
    
    fig_increase.update_traces(hoverlabel=dict(bgcolor='grey', font=dict(color='white')))

    # Calculate average values from the DataFrame for +intervention
    avg_x_increase = existing_data[stability_column_name].mean()
    avg_y_increase = existing_data[f'Normalized {capability_vector} ({base_column} +{intervention}%)'].mean()

    # Add average lines for +intervention
    fig_increase.add_shape(
        type='line',
        x0=avg_x_increase,
        x1=avg_x_increase,
        y0=fig_increase.data[0].y.min(),
        y1=fig_increase.data[0].y.max(),
        line=dict(color='red', width=2)
    )
    fig_increase.add_shape(
        type='line',
        x0=fig_increase.data[0].x.min(),
        x1=fig_increase.data[0].x.max(),
        y0=avg_y_increase,
        y1=avg_y_increase,
        line=dict(color='blue', width=2)
    )

    # Add labels for average lines for +intervention
    fig_increase.add_annotation(
        x=avg_x_increase,
        y=fig_increase.data[0].y.max(),
        text=f'Avg {stability_column_name}',
        showarrow=True,
        arrowhead=4,
        arrowcolor='red',
        ax=0,
        ay=-40,
        font=dict(color='red')
    )
    fig_increase.add_annotation(
        x=fig_increase.data[0].x.max(),
        y=avg_y_increase,
        text=f'Avg Normalized {capability_vector}',
        showarrow=True,
        arrowhead=4,
        arrowcolor='blue',
        ax=-40,
        ay=0,
        font=dict(color='blue')
    )

    # Add tooltips for +intervention
    fig_increase.update_traces(hoverinfo='text+name',
                               hovertext=["District: " + str(d) +
                                          f"<br>Impact ({capability_vector}): {y} <br>Stability ({base_column} +{intervention}%): {x}"
                                          for x, y, d in zip(fig_increase.data[0].x, fig_increase.data[0].y, existing_data['District'])])

    # Visualize with Plotly for -intervention
    fig_decrease = px.scatter(existing_data,
                              x=stability_column_name_opposite,
                              y=f'Impact Score {capability_vector} ({base_column} -{intervention}%)',
                              title=f'Impact vs Stability ({capability_vector} - {base_column} -{intervention}%)',
                              labels={'x': f'Stability ({base_column} -{intervention}%)', 'y': f'Impact ({capability_vector})'},
                              size_max=8,
                              width=800,
                              height=500)

    # Calculate average values from the DataFrame for -intervention
    avg_x_decrease = existing_data[stability_column_name_opposite].mean()
    avg_y_decrease = existing_data[f'Normalized {capability_vector} ({base_column} -{intervention}%)'].mean()

    # Add average lines for -intervention
    fig_decrease.add_shape(
        type='line',
        x0=avg_x_decrease,
        x1=avg_x_decrease,
        y0=fig_decrease.data[0].y.min(),
        y1=fig_decrease.data[0].y.max(),
        line=dict(color='red', width=2)
    )
    fig_decrease.add_shape(
        type='line',
        x0=fig_decrease.data[0].x.min(),
        x1=fig_decrease.data[0].x.max(),
        y0=avg_y_decrease,
        y1=avg_y_decrease,
        line=dict(color='blue', width=2)
    )

    # Add labels for average lines for -intervention
    fig_decrease.add_annotation(
        x=avg_x_decrease,
        y=fig_decrease.data[0].y.max(),
        text=f'Avg {stability_column_name_opposite}',
        showarrow=True,
        arrowhead=4,
        arrowcolor='red',
        ax=0,
        ay=-40,
        font=dict(color='red')
    )
    fig_decrease.add_annotation(
        x=fig_decrease.data[0].x.max(),
        y=avg_y_decrease,
        text=f'Avg Normalized {capability_vector}',
        showarrow=True,
        arrowhead=4,
        arrowcolor='blue',
        ax=-40,
        ay=0,
        font=dict(color='blue')
    )

    # Add tooltips for -intervention
    fig_decrease.update_traces(hoverinfo='text+name',
                               hovertext=["District: " + str(d) +
                                          f"<br>Impact ({capability_vector}): {y} <br>Stability ({base_column} -{intervention}%): {x}"
                                          for x, y, d in zip(fig_decrease.data[0].x, fig_decrease.data[0].y, existing_data['District'])])

    # Show the plots
    fig_increase.show()
    fig_decrease.show()

    return existing_data

base_Column = "NPK_Rice"
CapabilityVector = "Rice_Production"
adjacency_file = 'Karnataka_District_Adjacency_File - Copy.xlsx'
change_percentage = 20
inter = [10,20]
dim = 1

# Call the function and get the handles for the two plots
new_data_r = calc_and_vis_impact_stability_1D(G, existing_data.copy(), adjacency_file, base_Column, CapabilityVector, change_percentage, dim)

# Display the resulting DataFrame
new_data_r.head()


11.982189439376699 21969.740534991743


,District,Rice_Production (NPK_Rice +20%),Rice_Production (NPK_Rice -20%),Normalized Rice_Production (NPK_Rice +20%),Normalized Rice_Production (NPK_Rice -20%),Impact Score Rice_Production (NPK_Rice +20%),Impact Score Rice_Production (NPK_Rice -20%),New Stability(NPK + 20%),New Stability(NPK - 20%)
0,BENGALURU,29357.80982,26895.12006,0.01208,0.01208,0.01208,0.98792,0.99199,0.99199
1,BENGALURU(R),24382.98255,23578.56855,0.00382,0.00382,0.00382,0.99618,0.99370,0.99370
2,RAMANAGARA,25678.11639,24441.99110,0.00597,0.00597,0.00597,0.99403,0.89907,0.89907
3,CHITRADURGA,24601.50321,23724.24898,0.00418,0.00418,0.00418,0.99582,0.66904,0.66904
4,DAVANAGERE,326761.11825,225163.99235,0.50567,0.50567,0.50567,0.49433,0.78245,0.78245


In [39]:
base_Column = "TotalNPK"
CapabilityVector = "Rice_Production"
adjacency_file = 'Karnataka_District_Adjacency_File - Copy.xlsx'
change_percentage = 20
inter = [10,20]
dim = 1

# Call the function and get the handles for the two plots
new_data_1 = calc_and_vis_impact_stability_1D(G, existing_data.copy(), adjacency_file, base_Column, CapabilityVector, change_percentage, dim)

# Display the resulting DataFrame
new_data_1.head()


2.1764894064966205 17136.471837471167


,District,Rice_Production (TotalNPK +20%),Rice_Production (TotalNPK -20%),Normalized Rice_Production (TotalNPK +20%),Normalized Rice_Production (TotalNPK -20%),Impact Score Rice_Production (TotalNPK +20%),Impact Score Rice_Production (TotalNPK -20%),New Stability(NPK + 20%),New Stability(NPK - 20%)
0,BENGALURU,78017.23352,57723.64629,0.09872,0.09872,0.09872,0.90128,0.95972,0.95972
1,BENGALURU(R),67436.88321,50670.07942,0.07542,0.07542,0.07542,0.92458,0.97333,0.97333
2,RAMANAGARA,36651.74645,30146.65491,0.00762,0.00762,0.00762,0.99238,0.85307,0.85307
3,CHITRADURGA,110857.84687,79617.38853,0.17105,0.17105,0.17105,0.82895,0.82793,0.82793
4,DAVANAGERE,277385.40434,190635.76017,0.53782,0.53782,0.53782,0.46218,0.76111,0.76111


In [40]:
tf = pd.DataFrame({
    'District': new_data_r['District'],
    'Rice_Production': df['Rice_Production'],
    'Rice_Production (NPK_Rice +20%)': new_data_r['Rice_Production (NPK_Rice +20%)'],
    'Rice_Production (TotalNPK +20%)': new_data_1['Rice_Production (TotalNPK +20%)']   ,
    'Rice_Production (TotalNPK -20%)': new_data_1['Rice_Production (TotalNPK -20%)'],
    'Rice_Production (NPK_Rice -20%)': new_data_r['Rice_Production (NPK_Rice -20%)'],
})


In [41]:
tf

,District,Rice_Production,Rice_Production (NPK_Rice +20%),Rice_Production (TotalNPK +20%),Rice_Production (TotalNPK -20%),Rice_Production (NPK_Rice -20%)
0,BENGALURU,3376.00000,29357.80982,78017.23352,57723.64629,26895.12006
1,BENGALURU(R),2354.00000,24382.98255,67436.88321,50670.07942,23578.56855
2,RAMANAGARA,12128.00000,25678.11639,36651.74645,30146.65491,24441.99110
3,CHITRADURGA,4566.00000,24601.50321,110857.84687,79617.38853,23724.24898
4,DAVANAGERE,489505.00000,326761.11825,277385.40434,190635.76017,225163.99235
5,KOLAR,864.00000,23529.04448,75092.03175,55773.51178,23009.27650
6,CHIKKABALLAPURA,2698.00000,24984.08881,91198.92396,66511.43992,23979.30605
7,SHIVAMOGGA,384974.00000,276788.00063,164987.13841,115703.58289,191848.58060
8,TUMAKURU,26042.00000,37348.57993,116770.93329,83559.44614,32222.30013
9,CHIKKAMAGALURU,110533.00000,233253.57021,194380.19255,135298.95231,162825.62699


In [42]:
# ref = new_data[["District","Impact Score Rice_Production_223 (TotalNPK_315 +10%)","Impact Score Rice_Production_223 (TotalNPK_315 -10%)"]]

In [43]:
# cols_to_concat = ['Impact Score Rice_Production_223 (TotalNPK_315 +20%)', 'Impact Score Rice_Production_223 (TotalNPK_315 -20%)']
# res = pd.concat([ref, new_data_20[cols_to_concat]], axis=1)

In [44]:
# res.to_csv('res1.csv',index = False)

In [45]:
change_percentage = 10
dim = 1
base_Column = "NPK_Rice"
CapabilityVector = "Rice_Production"
# Call the function and get the handles for the two plots
new_data = calc_and_vis_impact_stability_1D(G, existing_data.copy(), adjacency_file, base_Column, CapabilityVector, change_percentage, dim)

# Display the resulting DataFrame
new_data.head()

11.982189439376699 21969.740534991743


,District,Rice_Production (NPK_Rice +10%),Rice_Production (NPK_Rice -10%),Normalized Rice_Production (NPK_Rice +10%),Normalized Rice_Production (NPK_Rice -10%),Impact Score Rice_Production (NPK_Rice +10%),Impact Score Rice_Production (NPK_Rice -10%),New Stability(NPK + 10%),New Stability(NPK - 10%)
0,BENGALURU,28742.13738,27510.79250,0.01208,0.01208,0.01208,0.98792,0.99199,0.99199
1,BENGALURU(R),24181.87905,23779.67205,0.00382,0.00382,0.00382,0.99618,0.99370,0.99370
2,RAMANAGARA,25369.08507,24751.02242,0.00597,0.00597,0.00597,0.99403,0.89907,0.89907
3,CHITRADURGA,24382.18965,23943.56254,0.00418,0.00418,0.00418,0.99582,0.66904,0.66904
4,DAVANAGERE,301361.83678,250563.27382,0.50567,0.50567,0.50567,0.49433,0.78245,0.78245


In [46]:

# base_Column = "TotalNPK"
# CapabilityVector = "Maize_Production"
# change_percentage = 20
# dim = 1

# # Call the function and get the handles for the two plots
# new_data = calc_and_vis_impact_stability_1D(G, existing_data.copy(), adjacency_file, base_Column, CapabilityVector, change_percentage, dim)

# # Display the resulting DataFrame
# new_data.head()

In [47]:
new_data

,District,Rice_Production (NPK_Rice +10%),Rice_Production (NPK_Rice -10%),Normalized Rice_Production (NPK_Rice +10%),Normalized Rice_Production (NPK_Rice -10%),Impact Score Rice_Production (NPK_Rice +10%),Impact Score Rice_Production (NPK_Rice -10%),New Stability(NPK + 10%),New Stability(NPK - 10%)
0,BENGALURU,28742.13738,27510.79250,0.01208,0.01208,0.01208,0.98792,0.99199,0.99199
1,BENGALURU(R),24181.87905,23779.67205,0.00382,0.00382,0.00382,0.99618,0.99370,0.99370
2,RAMANAGARA,25369.08507,24751.02242,0.00597,0.00597,0.00597,0.99403,0.89907,0.89907
3,CHITRADURGA,24382.18965,23943.56254,0.00418,0.00418,0.00418,0.99582,0.66904,0.66904
4,DAVANAGERE,301361.83678,250563.27382,0.50567,0.50567,0.50567,0.49433,0.78245,0.78245
5,KOLAR,23399.10248,23139.21849,0.00240,0.00240,0.00240,0.99760,0.99550,0.99550
6,CHIKKABALLAPURA,24732.89312,24230.50174,0.00482,0.00482,0.00482,0.99518,0.99430,0.99430
7,SHIVAMOGGA,255553.14562,213083.43561,0.42273,0.42273,0.42273,0.57727,0.82488,0.82488
8,TUMAKURU,36067.00998,33503.87008,0.02534,0.02534,0.02534,0.97466,0.88483,0.88483
9,CHIKKAMAGALURU,215646.58440,180432.61279,0.35048,0.35048,0.35048,0.64952,0.85158,0.85158


## 2D Capability vector

The Code to get the new production values and the stability of the perturbation for a 2-dimensional Capability vector and the visualize it.

In [48]:
res_df = pd.DataFrame()

In [49]:
import statsmodels.api as sm
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from itertools import cycle

def calc_and_vis_impact_stability_2D1(G, existing_data, adjacency_file, factor, base_column, capability_vector, change_percentages, dim):
    change_min_value = float('inf')
    change_max_value = float('-inf')
    
    initial_stability_column = calculate_and_return_initial_stability(df, capability_vector, G)
    existing_data = pd.concat([existing_data, initial_stability_column], axis=1)

    for intervention in change_percentages:
        for cap_col in capability_vector:
            df['temp'] = df[factor]*df[cap_col]
            X = sm.add_constant(df['temp'])
            y = df[cap_col]
            model = sm.OLS(y, X).fit()
            m, c = model.params['temp'], model.params['const']

            base_column_increase = (1 + intervention/100) * df['temp']
            base_column_decrease = (1 - intervention/100) * df['temp']

            change_vector_increase = m * base_column_increase + c
            change_vector_decrease = m * base_column_decrease + c
            
            change_min_value = min(change_min_value, change_vector_increase.min(), change_vector_decrease.min())
            change_max_value = max(change_max_value, change_vector_increase.max(), change_vector_decrease.max())
            
    df.drop('temp',axis =1)
    result_df = pd.DataFrame()
    res1_df = pd.DataFrame()
    
    for intervention in change_percentages:
        col_pass_plus = []
        col_pass_minus = []
        for cap_col in capability_vector:
            # Calculate Impact Score
            # filtered_df = df[(df[cap_col] != 0) & (~df[cap_col].isnull())]
            # filtered_df['temp'] = filtered_df[factor]*filtered_df[cap_col]
            df['temp'] = df[factor]*df[cap_col]
            X = sm.add_constant(df['temp'])
            y = df[cap_col]
            model = sm.OLS(y, X).fit()

            m, c = model.params['temp'], model.params['const']
            print(m,c,cap_col,intervention)
            
            base_column_increase = (1 + intervention/100) * df['temp']
            base_column_decrease = (1 - intervention/100) * df['temp']
            df.drop('temp',axis =1)
            
            new_vector_increase = m * base_column_increase + c
            new_vector_decrease = m * base_column_decrease + c
            
            if any(y == 0):
                new_vector_increase[y == 0] = 0
                new_vector_decrease[y == 0] = 0
                
            normalized_new_increase_vector = (new_vector_increase - change_min_value) / (change_max_value - change_min_value)
            normalized_new_decrease_vector = (new_vector_decrease - change_min_value) / (change_max_value - change_min_value)
            
            percent_change_inc = (new_vector_increase - (y) )*100/(y + 1e-9)
            percent_change_dec = (new_vector_decrease - (y) )*100/(y + 1e-9)
            
            result_df = pd.concat([result_df,
                pd.DataFrame({
                    f'{cap_col} ({base_column} +{intervention}%)': new_vector_increase,
                    f'{cap_col} ({base_column} -{intervention}%)': new_vector_decrease,
                    f'Normalized {cap_col} ({base_column} +{intervention}%)': normalized_new_increase_vector,
                    f'Normalized {cap_col} ({base_column} -{intervention}%)': normalized_new_decrease_vector,
                    f'Percent Change in {cap_col} ({base_column} +{intervention}%)': percent_change_inc,
                    f'Percent Change in {cap_col} ({base_column} -{intervention}%)': percent_change_dec,
                })
            ], axis=1)

            columns_to_pass = f'{cap_col} ({base_column} +{intervention}%)'
            col_pass_plus.append(columns_to_pass)
            
            # Calculate Stability for the opposite intervention and add to DataFrame
            opposite_intervention = -intervention
            columns_to_pass_opposite = f'{cap_col} ({base_column} -{intervention}%)'
            col_pass_minus.append(columns_to_pass_opposite)
        
        for column in result_df.columns:
            existing_data[column] = result_df[column]

        stability_column, stability_column_name = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, col_pass_plus, intervention, dim)

        capability_first_words = [col.split('_')[0] for col in capability_vector]
        stability_column_name_modified = stability_column_name + "_" + "_".join(capability_first_words)
        
        stability_column_opposite, stability_column_name_opposite = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, col_pass_minus, opposite_intervention, dim)

        stability_column_name_opposite_modified = stability_column_name_opposite + "_" + "_".join(capability_first_words)

        stress_column, stress_column_name = calculate_and_map_stress(G, existing_data.copy(), adjacency_file, col_pass_plus, intervention, dim)
        capability_first_words = [col.split('_')[0] for col in capability_vector]
        stress_column_name_modified = stress_column_name + "_" + "_".join(capability_first_words)       
         
        stress_column_opposite, stress_column_name_opposite = calculate_and_map_stress(G, existing_data.copy(), adjacency_file, col_pass_minus, opposite_intervention, dim)
        stress_column_name_opposite_modified = stress_column_name_opposite + "_" + "_".join(capability_first_words)

        res1_df = pd.concat([res1_df,
                pd.DataFrame({
                    stability_column_name_modified: stress_column,
                    stability_column_name_opposite_modified: stress_column_opposite,
                })
            ], axis=1)
        
    normalized_data_1 = res1_df.copy()
    min_value = normalized_data_1.min().min()
    max_value = normalized_data_1.max().max()
    normalized_data_1 = 1 - (normalized_data_1 - min_value) / (max_value - min_value)
    
    for column in normalized_data_1.columns:
        existing_data[column] = normalized_data_1[column]
        res1_df[column] = normalized_data_1[column]
        
        
    color_palette = ['red', 'blue', 'green', 'orange', 'purple', 'violet', 'yellow']
    color_cycle = cycle(color_palette)
    capability_color_map = {cap_col: next(color_cycle) for cap_col in capability_vector}
    fig = make_subplots(rows=len(change_percentages), cols=2, shared_xaxes=True, shared_yaxes=True, vertical_spacing=0.1, horizontal_spacing=0.1, row_heights=[0.5] * len(change_percentages))
    
    for idx, intervention in enumerate(change_percentages):
        for cap_col in capability_vector:
            color = capability_color_map[cap_col]

            fig.update_xaxes(title_text=f'Stability ({base_column} +{intervention}%)', row=idx + 1, col=1, showline=True, showgrid=True)
            fig.update_yaxes(title_text='Percent Change', row=idx + 1, col=1, showline=True, showgrid=True)
            fig.update_xaxes(title_text=f'Stability ({base_column} -{intervention}%)', row=idx + 1, col=2, showline=True, showgrid=True)
            fig.update_yaxes(title_text='Percent Change', row=idx + 3, col=2, showline=True, showgrid=True)
            
            fig.update_layout(margin=dict(t=50))
            
            scatter_increase = go.Scatter(
                x=existing_data[stability_column_name_modified],
                y=existing_data[f"Percent Change in {cap_col} ({base_column} +{abs(intervention)}%)"],
                mode='markers',
                marker=dict(size=5, symbol='circle', color=color),
                name=f'{cap_col} +{intervention}%',
                text=existing_data['District'],
                hovertemplate='%{text}' + '<br>Percent Change: %{y:.2f}' + '<br>Stability: %{x:.2f}',
                showlegend=False
            )

            fig.add_trace(scatter_increase, row=idx + 1, col=1)

            scatter_decrease = go.Scatter(
                x=existing_data[stability_column_name_opposite_modified],
                y=existing_data[
                    f"Percent Change in {cap_col} ({base_column} {'+' if intervention <= 0 else '-'}{abs(intervention)}%)"],
                mode='markers',
                marker=dict(size=5, symbol='circle', color=color),
                name=f'{cap_col} -{intervention}%',
                text=existing_data['District'],
                hovertemplate='%{text}' + '<br>Percent Change: %{y:.2f}' + '<br>Stability: %{x:.2f}',
                showlegend=False
            )

            fig.add_trace(scatter_decrease, row=idx + 1, col=2)

        avg_stability_increase = existing_data[stability_column_name_modified].mean()
        avg_stability_decrease = existing_data[stability_column_name_opposite_modified].mean()
        all_values = [
            existing_data[f"Percent Change in {capability_vector[0]} ({base_column} +{abs(intervention)}%)"].min(),
            existing_data[f"Percent Change in {capability_vector[0]} ({base_column} -{abs(intervention)}%)"].min(),
            existing_data[f"Percent Change in {capability_vector[0]} ({base_column} +{abs(intervention)}%)"].max(),
            existing_data[f"Percent Change in {capability_vector[0]} ({base_column} -{abs(intervention)}%)"].max()
        ]

        filtered_values = [value for value in all_values if not np.isinf(value)]

        # Calculate the range
        if filtered_values:
            y_range_1 = max(abs(min(filtered_values)), max(filtered_values))
        else:
            y_range_1 = 0  # If all values are inf or -inf, set the range to 0

        y_axis_1 = max(y_range_1, 1)
        print(y_axis_1)
        fig.add_shape(
            go.layout.Shape(
                type="line",
                x0=avg_stability_increase,
                x1=avg_stability_increase,
                y0=-10000,
                y1=y_axis_1,
                line=dict(color="black", dash="dash", width=1),
                name="Avg Stability"
            ),
            row=idx + 1, col=1
        )
        fig.add_shape(
            go.layout.Shape(
                type="line",
                x0=avg_stability_decrease,
                x1=avg_stability_decrease,
                y0=-10000,
                y1=y_axis_1,
                line=dict(color="black", dash="dash", width=1),
                name="Avg Stability"
            ),
            row=idx + 1, col=2
        )

        avg_impact_increase = existing_data[f"Percent Change in {capability_vector[0]} ({base_column} +{abs(intervention)}%)"].mean()
        avg_impact_decrease = existing_data[f"Percent Change in {capability_vector[0]} ({base_column} {'+' if intervention <= 0 else '-'}{abs(intervention)}%)"].mean()

        # Add shapes for average impact lines
        fig.add_shape(
            go.layout.Shape(
                type="line",
                x0=0,
                x1=1,
                y0=avg_impact_increase,
                y1=avg_impact_increase,
                line=dict(color="black", dash="dash", width=1),
                name="Avg Impact"
            ),
            row=idx + 1, col=1
        )
        fig.add_shape(
            go.layout.Shape(
                type="line",
                x0=0,
                x1=1,
                y0=avg_impact_decrease,
                y1=avg_impact_decrease,
                line=dict(color="black", dash="dash", width=1),
                name="Avg Impact"
            ),
            row=idx + 1, col=2
        )


    for cap_col in capability_vector:
        fig.add_trace(go.Scatter(x=[None], y=[None], mode='markers', marker=dict(size=5, symbol='circle', color=capability_color_map[cap_col]), name=cap_col), row=1, col=1)

    fig.update_layout(
        height=800,  
        showlegend=True,
        legend=dict(title="Capability", orientation="v", yanchor="bottom", y=0.85, xanchor="left", x=1),
    )
    fig.show()

    
    return existing_data, result_df, res1_df

factor = "FCR"
base_column = "TotalNPK"
capability_vector = ["Rice_Production", "Maize_Production"]
change_percentages = [10,20]
dim = len(capability_vector)

new_data_1,result_df,res1_df = calc_and_vis_impact_stability_2D1(G, existing_data.copy(), adjacency_file, factor, base_column, capability_vector, change_percentages, dim)
new_data_1.head()

11.9821894393767 21969.740534991743 Rice_Production 10
14.592486866377046 16260.762174034226 Maize_Production 10
11.9821894393767 21969.740534991743 Rice_Production 20
14.592486866377046 16260.762174034226 Maize_Production 20
25866.88334379366
25877.804232295293


,District,Initial Stability,Rice_Production (TotalNPK +10%),Rice_Production (TotalNPK -10%),Normalized Rice_Production (TotalNPK +10%),Normalized Rice_Production (TotalNPK -10%),Percent Change in Rice_Production (TotalNPK +10%),Percent Change in Rice_Production (TotalNPK -10%),Maize_Production (TotalNPK +10%),Maize_Production (TotalNPK -10%),...,Maize_Production (TotalNPK +20%),Maize_Production (TotalNPK -20%),Normalized Maize_Production (TotalNPK +20%),Normalized Maize_Production (TotalNPK -20%),Percent Change in Maize_Production (TotalNPK +20%),Percent Change in Maize_Production (TotalNPK -20%),New Stability(NPK + 10%)_Rice_Maize,New Stability(NPK - 10%)_Rice_Maize,New Stability(NPK + 20%)_Rice_Maize,New Stability(NPK - 20%)_Rice_Maize
0,BENGALURU,0.97384,28742.13738,27510.79250,0.01476,0.01330,751.36663,714.89314,23294.31502,22015.48723,...,23933.72892,21376.07334,0.00907,0.00605,731.32091,642.48258,0.99048,0.99369,0.98888,0.99530
1,BENGALURU(R),0.98397,24181.87905,23779.67205,0.00937,0.00889,927.26759,910.18148,64995.11946,56134.32722,...,69425.51557,51703.93111,0.06287,0.04192,63.03576,21.41918,0.99202,0.99495,0.99055,0.99641
2,RAMANAGARA,0.86835,25369.08507,24751.02242,0.01077,0.01004,109.17781,104.08165,19827.86325,19179.29942,...,20152.14516,18855.01750,0.00460,0.00307,92.84349,80.43079,0.90663,0.92509,0.89740,0.93431
3,CHITRADURGA,0.56271,24382.18965,23943.56254,0.00960,0.00909,433.99452,424.38814,191263.30935,159444.66441,...,207172.63182,143535.34194,0.22577,0.15052,-23.82659,-47.22480,0.74786,0.79519,0.72420,0.81885
4,DAVANAGERE,0.07259,301361.83678,250563.27382,0.33716,0.27709,-38.43539,-48.81293,428835.88026,353822.22243,...,466342.70918,316315.39351,0.53227,0.35485,-21.43081,-46.70734,0.63799,0.70529,0.60434,0.73894


In [50]:
new_data_1

,District,Initial Stability,Rice_Production (TotalNPK +10%),Rice_Production (TotalNPK -10%),Normalized Rice_Production (TotalNPK +10%),Normalized Rice_Production (TotalNPK -10%),Percent Change in Rice_Production (TotalNPK +10%),Percent Change in Rice_Production (TotalNPK -10%),Maize_Production (TotalNPK +10%),Maize_Production (TotalNPK -10%),...,Maize_Production (TotalNPK +20%),Maize_Production (TotalNPK -20%),Normalized Maize_Production (TotalNPK +20%),Normalized Maize_Production (TotalNPK -20%),Percent Change in Maize_Production (TotalNPK +20%),Percent Change in Maize_Production (TotalNPK -20%),New Stability(NPK + 10%)_Rice_Maize,New Stability(NPK - 10%)_Rice_Maize,New Stability(NPK + 20%)_Rice_Maize,New Stability(NPK - 20%)_Rice_Maize
0,BENGALURU,0.97384,28742.13738,27510.79250,0.01476,0.01330,751.36663,714.89314,23294.31502,22015.48723,...,23933.72892,21376.07334,0.00907,0.00605,731.32091,642.48258,0.99048,0.99369,0.98888,0.99530
1,BENGALURU(R),0.98397,24181.87905,23779.67205,0.00937,0.00889,927.26759,910.18148,64995.11946,56134.32722,...,69425.51557,51703.93111,0.06287,0.04192,63.03576,21.41918,0.99202,0.99495,0.99055,0.99641
2,RAMANAGARA,0.86835,25369.08507,24751.02242,0.01077,0.01004,109.17781,104.08165,19827.86325,19179.29942,...,20152.14516,18855.01750,0.00460,0.00307,92.84349,80.43079,0.90663,0.92509,0.89740,0.93431
3,CHITRADURGA,0.56271,24382.18965,23943.56254,0.00960,0.00909,433.99452,424.38814,191263.30935,159444.66441,...,207172.63182,143535.34194,0.22577,0.15052,-23.82659,-47.22480,0.74786,0.79519,0.72420,0.81885
4,DAVANAGERE,0.07259,301361.83678,250563.27382,0.33716,0.27709,-38.43539,-48.81293,428835.88026,353822.22243,...,466342.70918,316315.39351,0.53227,0.35485,-21.43081,-46.70734,0.63799,0.70529,0.60434,0.73894
5,KOLAR,0.91809,23399.10248,23139.21849,0.00844,0.00813,2608.22945,2578.15029,19804.71100,19160.35667,...,20126.88817,18838.17950,0.00457,0.00305,1044.22332,970.95961,0.92421,0.93947,0.91658,0.94710
6,CHIKKABALLAPURA,0.88384,24732.89312,24230.50174,0.01002,0.00943,816.71212,798.09124,147835.18114,123912.55951,...,159796.49195,111951.24869,0.16975,0.11316,51.47879,6.12398,0.86648,0.89224,0.85361,0.90512
7,SHIVAMOGGA,0.63073,255553.14562,213083.43561,0.28299,0.23277,-33.61808,-44.64992,183652.21335,153217.40405,...,198869.61801,137999.99940,0.21596,0.14397,-12.21125,-39.08145,0.86893,0.89424,0.85628,0.90690
8,TUMAKURU,0.86588,36067.00998,33503.87008,0.02342,0.02039,38.49555,28.65321,60343.86239,52328.75326,...,64351.41695,48321.19869,0.05687,0.03792,-3.76351,-27.73644,0.87528,0.89944,0.86321,0.91152
9,CHIKKAMAGALURU,0.69899,215646.58440,180432.61279,0.23580,0.19415,95.09702,63.23868,147183.37828,123379.26626,...,159085.43429,111477.21025,0.16891,0.11260,159.29528,81.69806,0.88802,0.91000,0.87703,0.92099


In [51]:
# result_df_2

In [52]:
abbreviation_mapping = {
    'BENGALURU': 'BLR',
    'BENGALURU(R)': 'BLR(R)',
    'RAMANAGARA': 'RGA',
    'CHITRADURGA': 'CDA',
    'DAVANAGERE': 'DVG',
    'KOLAR': 'KLR',
    'CHIKKABALLAPURA': 'CKA',
    'SHIVAMOGGA': 'SMG',
    'TUMAKURU': 'TKR',
    'CHIKKAMAGALURU': 'CMG',
    'DAKSHINA KANNADA': 'DKA',
    'UDUPI': 'UPI',
    'HASSAN': 'HSN',
    'KODAGU': 'KDG',
    'MANDYA': 'MDY',
    'MYSURU': 'MYS',
    'CHAMARAJANAGAR': 'CNR',
    'BELAGAVI': 'BLG',
    'VIJAYAPURA': 'VJP',
    'BAGALKOT': 'BKT',
    'DHARAWAD': 'DWD',
    'GADAG': 'GDG',
    'HAVERI': 'HVR',
    'UTTARA KANNADA': 'UTK',
    'BALLARI': 'BLL',
    'BIDAR': 'BDR',
    'KALABURAGI': 'KLB',
    'YADGIRI': 'YDR',
    'RAICHUR': 'RCR',
    'KOPPAL': 'KPL',
    'Vijayanagara': 'VNG',
}

df['Abbreviation'] = df['District'].map(abbreviation_mapping)

In [53]:
df['Abbreviation']

0        BLR
1     BLR(R)
2        RGA
3        CDA
4        DVG
5        KLR
6        CKA
7        SMG
8        TKR
9        CMG
10       DKA
11       UPI
12       HSN
13       KDG
14       MDY
15       MYS
16       CNR
17       BLG
18       VJP
19       BKT
20       DWD
21       GDG
22       HVR
23       UTK
24       BLL
25       BDR
26       KLB
27       YDR
28       RCR
29       KPL
30       VNG
Name: Abbreviation, dtype: object

In [54]:
# dft = pd.read_excel('Rice_Maize_10_20_31.xlsx')

In [55]:
# repeated_column = np.tile(df['Abbreviation'], 8)

# # Create a DataFrame with the repeated column
# repeated_df = pd.DataFrame(repeated_column, columns=['Abbreviations'])

# # Concatenate the repeated column DataFrame with dft along axis=1 (columns)
# result_df_2 = pd.concat([result_df_2, repeated_df], axis=1)

In [56]:
# result_df_2

In [57]:
# result_df_2.to_excel('Rice_Maize_10_20_31.xlsx')

In [58]:
res1_df.columns = [col.replace('_Rice_Maize', '') for col in res1_df.columns]

print(res1_df.columns)

Index(['New Stability(NPK + 10%)', 'New Stability(NPK - 10%)',
       'New Stability(NPK + 20%)', 'New Stability(NPK - 20%)'],
      dtype='object')


In [59]:
res1_df

,New Stability(NPK + 10%),New Stability(NPK - 10%),New Stability(NPK + 20%),New Stability(NPK - 20%)
0,0.99048,0.99369,0.98888,0.99530
1,0.99202,0.99495,0.99055,0.99641
2,0.90663,0.92509,0.89740,0.93431
3,0.74786,0.79519,0.72420,0.81885
4,0.63799,0.70529,0.60434,0.73894
5,0.92421,0.93947,0.91658,0.94710
6,0.86648,0.89224,0.85361,0.90512
7,0.86893,0.89424,0.85628,0.90690
8,0.87528,0.89944,0.86321,0.91152
9,0.88802,0.91000,0.87703,0.92099


In [60]:
new_data_1.columns

Index(['District', 'Initial Stability', 'Rice_Production (TotalNPK +10%)',
       'Rice_Production (TotalNPK -10%)',
       'Normalized Rice_Production (TotalNPK +10%)',
       'Normalized Rice_Production (TotalNPK -10%)',
       'Percent Change in Rice_Production (TotalNPK +10%)',
       'Percent Change in Rice_Production (TotalNPK -10%)',
       'Maize_Production (TotalNPK +10%)', 'Maize_Production (TotalNPK -10%)',
       'Normalized Maize_Production (TotalNPK +10%)',
       'Normalized Maize_Production (TotalNPK -10%)',
       'Percent Change in Maize_Production (TotalNPK +10%)',
       'Percent Change in Maize_Production (TotalNPK -10%)',
       'Rice_Production (TotalNPK +20%)', 'Rice_Production (TotalNPK -20%)',
       'Normalized Rice_Production (TotalNPK +20%)',
       'Normalized Rice_Production (TotalNPK -20%)',
       'Percent Change in Rice_Production (TotalNPK +20%)',
       'Percent Change in Rice_Production (TotalNPK -20%)',
       'Maize_Production (TotalNPK +20%)', 'M

Adding the Relative Change columns to the dataframe

In [61]:
ch = [-10, 10, 20, -20]

for intervention in ch:
    for cap_col in capability_vector:
        percent_change_col = new_data_1[f"Percent Change in {cap_col} (TotalNPK {'+' if intervention >= 0 else '-'}{abs(intervention)}%)"]
        percent_change_col.replace([np.inf, -np.inf], np.nan, inplace=True)
    
        min_val = percent_change_col.min()
        max_val = percent_change_col.max()
        
        if np.isnan(min_val) or np.isnan(max_val):
            continue
        
        normalized_col = (percent_change_col - min_val) / (max_val - min_val)
        relative_col = percent_change_col/100
        impact_col_name = f"Impact in {cap_col} (TotalNPK {'+' if intervention >= 0 else '-'}{abs(intervention)}%)"
        relative_col_name = f"Relative Change in {cap_col} (TotalNPK {'+' if intervention >= 0 else '-'}{abs(intervention)}%)"
        new_data_1[impact_col_name] = normalized_col
        new_data_1[relative_col_name] = relative_col


In [62]:
new_data_1.columns

Index(['District', 'Initial Stability', 'Rice_Production (TotalNPK +10%)',
       'Rice_Production (TotalNPK -10%)',
       'Normalized Rice_Production (TotalNPK +10%)',
       'Normalized Rice_Production (TotalNPK -10%)',
       'Percent Change in Rice_Production (TotalNPK +10%)',
       'Percent Change in Rice_Production (TotalNPK -10%)',
       'Maize_Production (TotalNPK +10%)', 'Maize_Production (TotalNPK -10%)',
       'Normalized Maize_Production (TotalNPK +10%)',
       'Normalized Maize_Production (TotalNPK -10%)',
       'Percent Change in Maize_Production (TotalNPK +10%)',
       'Percent Change in Maize_Production (TotalNPK -10%)',
       'Rice_Production (TotalNPK +20%)', 'Rice_Production (TotalNPK -20%)',
       'Normalized Rice_Production (TotalNPK +20%)',
       'Normalized Rice_Production (TotalNPK -20%)',
       'Percent Change in Rice_Production (TotalNPK +20%)',
       'Percent Change in Rice_Production (TotalNPK -20%)',
       'Maize_Production (TotalNPK +20%)', 'M

In [63]:
new_data_1['Relative Change in Rice_Production (TotalNPK +10%)']

0      7.51367
1      9.27268
2      1.09178
3      4.33995
4     -0.38435
5     26.08229
6      8.16712
7     -0.33618
8      0.38496
9      0.95097
10     0.05029
11    -0.56227
12    -0.17458
13     1.84271
14     0.07411
15    -0.27054
16     0.12880
17     0.22726
18   128.84059
19   258.66883
20     0.58404
21     2.55915
22    -0.13213
23    -0.29519
24     0.17123
25    10.96833
26     2.42535
27     0.91542
28     0.16664
29    -0.14788
30     0.17122
Name: Relative Change in Rice_Production (TotalNPK +10%), dtype: float64

In [64]:
newdf = pd.DataFrame()
newdf = pd.concat([newdf, new_data_1['District']], axis=1) 
newdf = pd.concat([newdf, new_data_1['Percent Change in Rice_Production (TotalNPK +10%)']], axis=1) 
newdf = pd.concat([newdf, new_data_1['Relative Change in Rice_Production (TotalNPK +10%)']], axis=1) 
newdf = pd.concat([newdf, new_data_1['Impact in Rice_Production (TotalNPK +10%)']], axis=1) 
newdf = pd.concat([newdf, new_data_1['Percent Change in Maize_Production (TotalNPK +10%)']], axis=1) 
newdf = pd.concat([newdf, new_data_1['Relative Change in Maize_Production (TotalNPK +10%)']], axis=1) 
newdf = pd.concat([newdf, new_data_1['Impact in Maize_Production (TotalNPK +10%)']], axis=1) 
newdf.columns

Index(['District', 'Percent Change in Rice_Production (TotalNPK +10%)',
       'Relative Change in Rice_Production (TotalNPK +10%)',
       'Impact in Rice_Production (TotalNPK +10%)',
       'Percent Change in Maize_Production (TotalNPK +10%)',
       'Relative Change in Maize_Production (TotalNPK +10%)',
       'Impact in Maize_Production (TotalNPK +10%)'],
      dtype='object')

In [65]:
newdf.to_csv('test1.csv')

In [66]:
newdf

,District,Percent Change in Rice_Production (TotalNPK +10%),Relative Change in Rice_Production (TotalNPK +10%),Impact in Rice_Production (TotalNPK +10%),Percent Change in Maize_Production (TotalNPK +10%),Relative Change in Maize_Production (TotalNPK +10%),Impact in Maize_Production (TotalNPK +10%)
0,BENGALURU,751.36663,7.51367,0.03115,709.11132,7.09111,0.04964
1,BENGALURU(R),927.26759,9.27268,0.03794,52.63161,0.52632,0.00553
2,RAMANAGARA,109.17781,1.09178,0.00638,89.74032,0.89740,0.00802
3,CHITRADURGA,433.99452,4.33995,0.01891,-29.67614,-0.29676,0.00000
4,DAVANAGERE,-38.43539,-0.38435,0.00069,-27.74994,-0.27750,0.00013
5,KOLAR,2608.22945,26.08229,0.10278,1025.90739,10.25907,0.07093
6,CHIKKABALLAPURA,816.71212,8.16712,0.03367,40.14009,0.40140,0.00469
7,SHIVAMOGGA,-33.61808,-0.33618,0.00087,-18.92880,-0.18929,0.00072
8,TUMAKURU,38.49555,0.38496,0.00365,-9.75674,-0.09757,0.00134
9,CHIKKAMAGALURU,95.09702,0.95097,0.00584,139.89598,1.39896,0.01139


In [67]:
import pandas as pd
import numpy as np

ch = [-10, 10, 20, -20]
art = pd.DataFrame() 
art = pd.concat([art, new_data_1['District']], axis=1) 

for intervention in ch:
    all_impact_cols = [f"Impact in {cap_col} (TotalNPK {'+' if intervention >= 0 else '-'}{abs(intervention)}%)" for cap_col in capability_vector]
    all_relative_change_cols = [f"Relative Change in {cap_col} (TotalNPK {'+' if intervention >= 0 else '-'}{abs(intervention)}%)" for cap_col in capability_vector]
    all_impacts = new_data_1[all_impact_cols]
    all_relative = new_data_1[all_relative_change_cols]
    art = pd.concat([art, all_impacts], axis=1) 
    art = pd.concat([art, all_relative], axis=1) 
    df = pd.concat([df, all_relative], axis=1)
     
# Replace inf values with 0
# art.replace([np.inf, -np.inf], 0, inplace=True)

# if art.isnull().values.any():
#     print("Warning: There are NaN values present after replacing inf.")

In [68]:
df

,District,TotalNPK,Rice_Production,Jowar_Production,Maize_Production,Jowar_Yield,TotalCerealsandMinorMillets_Production,TotalOilSeeds_Production,TotalFoodGrains,TotalGourdVarietyVegetables,...,temp,Abbreviation,Relative Change in Rice_Production (TotalNPK -10%),Relative Change in Maize_Production (TotalNPK -10%),Relative Change in Rice_Production (TotalNPK +10%),Relative Change in Maize_Production (TotalNPK +10%),Relative Change in Rice_Production (TotalNPK +20%),Relative Change in Maize_Production (TotalNPK +20%),Relative Change in Rice_Production (TotalNPK -20%),Relative Change in Maize_Production (TotalNPK -20%)
0,BENGALURU,23310.00000,3376.00000,851,2879,1539.00000,69153.00000,162.00000,72752.00000,11088.00000,...,438.18021,BLR,7.14893,6.64692,7.51367,7.09111,7.69603,7.31321,6.96656,6.42483
1,BENGALURU(R),19259.00000,2354.00000,0,42583,0.00000,125232.00000,479.00000,131795.00000,12614.00000,...,3036.08025,BLR(R),9.10181,0.31823,9.27268,0.52632,9.35811,0.63036,9.01638,0.21419
2,RAMANAGARA,7472.00000,12128.00000,0,10450,0.00000,159008.00000,6414.00000,175873.00000,10071.00000,...,222.22526,RGA,1.04082,0.83534,1.09178,0.89740,1.11726,0.92843,1.01534,0.80431
3,CHITRADURGA,35884.00000,4566.00000,13301,271975,984.00000,388849.00000,78257.00000,421955.00000,6113.00000,...,10902.40657,CDA,4.24388,-0.41375,4.33995,-0.29676,4.38798,-0.23827,4.19585,-0.47225
4,DAVANAGERE,99644.00000,489505.00000,23072,593544,2167.00000,1126753.00000,21327.00000,1141132.00000,11822.00000,...,25702.83548,DVG,-0.48813,-0.40388,-0.38435,-0.27750,-0.33247,-0.21431,-0.54002,-0.46707
5,KOLAR,22190.00000,864.00000,0,1759,0.00000,69089.00000,5994.00000,87015.00000,14692.00000,...,220.78291,KLR,25.78150,9.89276,26.08229,10.25907,26.23269,10.44223,25.63111,9.70960
6,CHIKKABALLAPURA,28357.00000,2698.00000,0,105491,0.00000,165723.00000,7341.00000,171962.00000,19918.00000,...,8196.89675,CKA,7.98091,0.17463,8.16712,0.40140,8.26023,0.51479,7.88781,0.06124
7,SHIVAMOGGA,56609.00000,384974.00000,756,226532,2242.00000,613331.00000,2146.00000,613998.00000,238.00000,...,10428.24626,SMG,-0.44650,-0.32364,-0.33618,-0.18929,-0.28102,-0.12211,-0.50166,-0.39081
8,TUMAKURU,38148.00000,26042.00000,2652,66868,1346.00000,431562.00000,42331.00000,450563.00000,4382.00000,...,2746.31363,TKR,0.28653,-0.21743,0.38496,-0.09757,0.43417,-0.03764,0.23732,-0.27736
9,CHIKKAMAGALURU,67863.00000,110533.00000,9080,61353,897.00000,235695.00000,8612.00000,253087.00000,13083.00000,...,8156.29037,CMG,0.63239,1.01097,0.95097,1.39896,1.11026,1.59295,0.47310,0.81698


In [69]:
column_order = ['District','Impact in Rice_Production (TotalNPK +10%)','Impact in Rice_Production (TotalNPK -10%)','Impact in Maize_Production (TotalNPK +10%)',
    'Impact in Maize_Production (TotalNPK -10%)','Impact in Rice_Production (TotalNPK +20%)','Impact in Rice_Production (TotalNPK -20%)',
    'Impact in Maize_Production (TotalNPK +20%)','Impact in Maize_Production (TotalNPK -20%)']

art = art[column_order]
art.head()

,District,Impact in Rice_Production (TotalNPK +10%),Impact in Rice_Production (TotalNPK -10%),Impact in Maize_Production (TotalNPK +10%),Impact in Maize_Production (TotalNPK -10%),Impact in Rice_Production (TotalNPK +20%),Impact in Rice_Production (TotalNPK -20%),Impact in Maize_Production (TotalNPK +20%),Impact in Maize_Production (TotalNPK -20%)
0,BENGALURU,0.03115,0.02996,0.04964,0.04743,0.03175,0.02936,0.05075,0.04632
1,BENGALURU(R),0.03794,0.03750,0.00553,0.00492,0.03816,0.03728,0.00584,0.00461
2,RAMANAGARA,0.00638,0.00638,0.00802,0.00839,0.00638,0.00639,0.00784,0.00857
3,CHITRADURGA,0.01891,0.01875,0.00000,0.00000,0.01899,0.01867,0.00000,0.00000
4,DAVANAGERE,0.00069,0.00048,0.00013,0.00007,0.00079,0.00038,0.00016,0.00003


In [70]:
# act = pd.read_excel("Rice_Maize_10_20_31.xlsx")

In [71]:
art

,District,Impact in Rice_Production (TotalNPK +10%),Impact in Rice_Production (TotalNPK -10%),Impact in Maize_Production (TotalNPK +10%),Impact in Maize_Production (TotalNPK -10%),Impact in Rice_Production (TotalNPK +20%),Impact in Rice_Production (TotalNPK -20%),Impact in Maize_Production (TotalNPK +20%),Impact in Maize_Production (TotalNPK -20%)
0,BENGALURU,0.03115,0.02996,0.04964,0.04743,0.03175,0.02936,0.05075,0.04632
1,BENGALURU(R),0.03794,0.03750,0.00553,0.00492,0.03816,0.03728,0.00584,0.00461
2,RAMANAGARA,0.00638,0.00638,0.00802,0.00839,0.00638,0.00639,0.00784,0.00857
3,CHITRADURGA,0.01891,0.01875,0.00000,0.00000,0.01899,0.01867,0.00000,0.00000
4,DAVANAGERE,0.00069,0.00048,0.00013,0.00007,0.00079,0.00038,0.00016,0.00003
5,KOLAR,0.10278,0.10188,0.07093,0.06923,0.10323,0.10143,0.07178,0.06838
6,CHIKKABALLAPURA,0.03367,0.03317,0.00469,0.00395,0.03392,0.03292,0.00506,0.00358
7,SHIVAMOGGA,0.00087,0.00064,0.00072,0.00061,0.00099,0.00053,0.00078,0.00055
8,TUMAKURU,0.00365,0.00347,0.00134,0.00132,0.00374,0.00338,0.00135,0.00131
9,CHIKKAMAGALURU,0.00584,0.00481,0.01139,0.00957,0.00635,0.00429,0.01231,0.00866


In [72]:
capability_vector = ["Rice_Production", "Maize_Production"]

for intervention in ch:
    all_percent_change_cols = [f"Impact in {cap_col} (TotalNPK {'+' if intervention >= 0 else '-'}{abs(intervention)}%)" for cap_col in capability_vector]
    print(all_percent_change_cols)

['Impact in Rice_Production (TotalNPK -10%)', 'Impact in Maize_Production (TotalNPK -10%)']
['Impact in Rice_Production (TotalNPK +10%)', 'Impact in Maize_Production (TotalNPK +10%)']
['Impact in Rice_Production (TotalNPK +20%)', 'Impact in Maize_Production (TotalNPK +20%)']
['Impact in Rice_Production (TotalNPK -20%)', 'Impact in Maize_Production (TotalNPK -20%)']


Calculating the Dissonance between the two new production values for all the districts for each perturbation

In [73]:
capability_vector = ["Rice_Production", "Maize_Production"]

dissonance = pd.DataFrame(columns=capability_vector)

# List to store dissonance for each intervention
dis_list = []

# List to store mean for each intervention
mean_list = []

for intervention in ch:
    all_percent_change_cols = [f"Impact in {cap_col} (TotalNPK {'+' if intervention >= 0 else '-'}{abs(intervention)}%)" for cap_col in capability_vector]
    
    # Extracting columns for current intervention
    intervention_df = art.loc[:, all_percent_change_cols] 
      
    # Calculating maximum and minimum values for each row
    max_values = intervention_df.max(axis=1)
    min_values = intervention_df.min(axis=1)
    
    # Calculating differences for each row
    diff = max_values - min_values
    
    # Appending differences to the dissonance list
    dis_list.append(diff)
    
    # Calculating mean for each intervention
    mean = intervention_df.mean(axis=1)
    
    # Appending mean to the mean list
    mean_list.append(mean)

# Creating a DataFrame from the list of differences (dissonance)
dissonance = pd.DataFrame(dis_list).T
dissonance.columns = [f'Dis_{intervention}' for intervention in ch]

print("Dissonance:")
print(dissonance)

Dissonance:
    Dis_-10  Dis_10  Dis_20  Dis_-20
0   0.01747 0.01849 0.01900  0.01696
1   0.03258 0.03241 0.03232  0.03267
2   0.00201 0.00164 0.00146  0.00219
3   0.01875 0.01891 0.01899  0.01867
4   0.00042 0.00056 0.00063  0.00034
5   0.03265 0.03185 0.03145  0.03305
6   0.02922 0.02898 0.02886  0.02934
7   0.00004 0.00015 0.00021  0.00002
8   0.00215 0.00232 0.00240  0.00207
9   0.00476 0.00556 0.00596  0.00437
10  0.00084 0.00037 0.00097  0.00144
11  1.00000 1.00000 1.00000  1.00000
12  0.00033 0.00039 0.00042  0.00030
13  0.01182 0.01327 0.01400  0.01109
14  0.00633 0.00652 0.00661  0.00623
15  0.00047 0.00041 0.00038  0.00050
16  0.00126 0.00134 0.00137  0.00122
17  0.00117 0.00145 0.00159  0.00103
18  0.49867 0.49846 0.49835  0.49877
19  0.99546 0.99446 0.99396  0.99596
20  0.00042 0.00027 0.00019  0.00049
21  0.01146 0.01162 0.01170  0.01138
22  0.00041 0.00045 0.00047  0.00038
23  0.00352 0.00337 0.00330  0.00359
24  0.00196 0.00221 0.00233  0.00183
25  0.00080 0.00042 0.0002

Calculating the Mean between the two new production values for all the districts for each perturbation

In [74]:
# Creating a DataFrame from the list of means
mean_df = pd.DataFrame(mean_list).T
mean_df.columns = [f'Mean_{intervention}' for intervention in ch]

print("\nMean:")
print(mean_df)


Mean:
    Mean_-10  Mean_10  Mean_20  Mean_-20
0    0.03869  0.04040  0.04125   0.03784
1    0.02121  0.02173  0.02200   0.02094
2    0.00739  0.00720  0.00711   0.00748
3    0.00937  0.00946  0.00950   0.00933
4    0.00027  0.00041  0.00047   0.00021
5    0.08556  0.08686  0.08751   0.08491
6    0.01856  0.01918  0.01949   0.01825
7    0.00062  0.00080  0.00088   0.00054
8    0.00240  0.00250  0.00255   0.00234
9    0.00719  0.00862  0.00933   0.00647
10   0.00236  0.00218  0.00209   0.00245
11   0.50000  0.50000  0.50000   0.50000
12   0.00106  0.00130  0.00142   0.00093
13   0.01357  0.01591  0.01709   0.01240
14   0.00512  0.00571  0.00601   0.00483
15   0.00109  0.00133  0.00145   0.00097
16   0.00178  0.00200  0.00211   0.00167
17   0.00305  0.00377  0.00413   0.00269
18   0.24994  0.24995  0.24996   0.24993
19   0.50227  0.50277  0.50302   0.50202
20   0.00375  0.00429  0.00456   0.00349
21   0.00614  0.00623  0.00627   0.00610
22   0.00117  0.00143  0.00157   0.00103
23   0.00

Calculating the Sustainability scores of the perturbations for each district

In [75]:
# Create a new DataFrame for Sustainability scores
sustainability_scores = pd.DataFrame(index=res1_df.index)
sustainability_scores['District'] = df['Abbreviation']
for intervention in ch:
    # Extracting mean and dissonance columns for the current intervention
    mean_col = f'Mean_{intervention}'
    dissonance_col = f'Dis_{intervention}'
    stability_col = f'New Stability(NPK {"+" if intervention >= 0 else "-"} {abs(intervention)}%)'  # Adjusted stability column name
    
    # Calculate Sustainability score for the current intervention
    sustainability_score_col = (mean_df[mean_col] * res1_df[stability_col]) / dissonance[dissonance_col]
    
    # Add the calculated Sustainability score to the new DataFrame
    sustainability_scores[f'Sustainability_{intervention}'] = sustainability_score_col

In [76]:
# sustainability_scores.to_excel("Sustainability_Scores.xlsx",index=False)

In [77]:
sustainability_scores = pd.read_csv("Sustainability_Scores.csv")

In [78]:
sustainability_scores

,District,Sustainability_-10,Sustainability_10,Sustainability_20,Sustainability_-20,KGISDist 1
0,BLR,0.70609,0.68765,0.68011,0.71789,Bengaluru (Urban)
1,BLR(R),0.68471,0.69583,0.69997,0.67728,Bengaluru (Rural)
2,RGA,1.11829,0.92113,0.85832,1.29588,Ramanagara
3,CDA,0.42109,0.40276,0.39360,0.43025,Chitradurga
4,DVG,0.93611,0.98328,0.99518,0.89880,Davanagere
5,KLR,1.33098,1.25748,1.23121,1.38497,Kolara
6,CKA,0.56619,0.56782,0.56833,0.56494,Chikkaballapura
7,SMG,0.56883,0.57701,0.57969,0.56275,Shivamogga
8,TKR,1.91176,1.84162,1.81533,1.96066,Tumakuru
9,CMG,0.53083,0.51595,0.50837,0.53808,Chikkamagaluru


In [79]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=4, cols=1, subplot_titles=['NPK Fertilizers -10%', 'NPK Fertilizers +10%', 'NPK Fertilizers +20%', 'NPK Fertilizers -20%'])

for i, col in enumerate(['Sustainability_-10', 'Sustainability_10', 'Sustainability_20', 'Sustainability_-20'], start=1):
    fig.add_trace(go.Scatter(x=sustainability_scores['District'], y=sustainability_scores[col], mode='lines+markers', name=col, showlegend=False), row=i, col=1)

fig.update_xaxes(title_text="District", row=4, col=1)
fig.update_yaxes(title_text="SI Score", col=1)

fig.update_layout(height=800, width=1000, title_text="District-wise Sustainable Intervention Scores in SDG 2 Capability Indicators (NPK Fertilizer Perturbations)")

fig.show()


In [80]:
normalized_data_1 = res1_df.copy()
min_value = normalized_data_1.min().min()
max_value = normalized_data_1.max().max()
normalized_data_1 = (normalized_data_1 - min_value) / (max_value - min_value)
normalized_data_2 = 1 - (normalized_data_1 - min_value) / (max_value - min_value)

In [81]:
normalized_data_1

,New Stability(NPK + 10%),New Stability(NPK - 10%),New Stability(NPK + 20%),New Stability(NPK - 20%)
0,0.99048,0.99369,0.98888,0.99530
1,0.99202,0.99495,0.99055,0.99641
2,0.90663,0.92509,0.89740,0.93431
3,0.74786,0.79519,0.72420,0.81885
4,0.63799,0.70529,0.60434,0.73894
5,0.92421,0.93947,0.91658,0.94710
6,0.86648,0.89224,0.85361,0.90512
7,0.86893,0.89424,0.85628,0.90690
8,0.87528,0.89944,0.86321,0.91152
9,0.88802,0.91000,0.87703,0.92099


In [82]:
normalized_data_2

,New Stability(NPK + 10%),New Stability(NPK - 10%),New Stability(NPK + 20%),New Stability(NPK - 20%)
0,0.00952,0.00631,0.01112,0.00470
1,0.00798,0.00505,0.00945,0.00359
2,0.09337,0.07491,0.10260,0.06569
3,0.25214,0.20481,0.27580,0.18115
4,0.36201,0.29471,0.39566,0.26106
5,0.07579,0.06053,0.08342,0.05290
6,0.13352,0.10776,0.14639,0.09488
7,0.13107,0.10576,0.14372,0.09310
8,0.12472,0.10056,0.13679,0.08848
9,0.11198,0.09000,0.12297,0.07901


In [83]:
base_column = "TotalNPK"
capability_vector = ["Rice_Production", "Maize_Production"]
# change_percentage = 20
change_percentages = [10,20]
dim = len(capability_vector)

new_data_1,result_df,res1_df = calc_and_vis_impact_stability_2D1(G, existing_data.copy(), adjacency_file,factor, base_column, capability_vector, change_percentages, dim)
new_data_1.head()

11.9821894393767 21969.740534991743 Rice_Production 10
14.592486866377046 16260.762174034226 Maize_Production 10
11.9821894393767 21969.740534991743 Rice_Production 20
14.592486866377046 16260.762174034226 Maize_Production 20
25866.88334379366
25877.804232295293


,District,Initial Stability,Rice_Production (TotalNPK +10%),Rice_Production (TotalNPK -10%),Normalized Rice_Production (TotalNPK +10%),Normalized Rice_Production (TotalNPK -10%),Percent Change in Rice_Production (TotalNPK +10%),Percent Change in Rice_Production (TotalNPK -10%),Maize_Production (TotalNPK +10%),Maize_Production (TotalNPK -10%),...,Maize_Production (TotalNPK +20%),Maize_Production (TotalNPK -20%),Normalized Maize_Production (TotalNPK +20%),Normalized Maize_Production (TotalNPK -20%),Percent Change in Maize_Production (TotalNPK +20%),Percent Change in Maize_Production (TotalNPK -20%),New Stability(NPK + 10%)_Rice_Maize,New Stability(NPK - 10%)_Rice_Maize,New Stability(NPK + 20%)_Rice_Maize,New Stability(NPK - 20%)_Rice_Maize
0,BENGALURU,0.97384,28742.13738,27510.79250,0.01476,0.01330,751.36663,714.89314,23294.31502,22015.48723,...,23933.72892,21376.07334,0.00907,0.00605,731.32091,642.48258,0.99048,0.99369,0.98888,0.99530
1,BENGALURU(R),0.98397,24181.87905,23779.67205,0.00937,0.00889,927.26759,910.18148,64995.11946,56134.32722,...,69425.51557,51703.93111,0.06287,0.04192,63.03576,21.41918,0.99202,0.99495,0.99055,0.99641
2,RAMANAGARA,0.86835,25369.08507,24751.02242,0.01077,0.01004,109.17781,104.08165,19827.86325,19179.29942,...,20152.14516,18855.01750,0.00460,0.00307,92.84349,80.43079,0.90663,0.92509,0.89740,0.93431
3,CHITRADURGA,0.56271,24382.18965,23943.56254,0.00960,0.00909,433.99452,424.38814,191263.30935,159444.66441,...,207172.63182,143535.34194,0.22577,0.15052,-23.82659,-47.22480,0.74786,0.79519,0.72420,0.81885
4,DAVANAGERE,0.07259,301361.83678,250563.27382,0.33716,0.27709,-38.43539,-48.81293,428835.88026,353822.22243,...,466342.70918,316315.39351,0.53227,0.35485,-21.43081,-46.70734,0.63799,0.70529,0.60434,0.73894


In [84]:
new_data_1

,District,Initial Stability,Rice_Production (TotalNPK +10%),Rice_Production (TotalNPK -10%),Normalized Rice_Production (TotalNPK +10%),Normalized Rice_Production (TotalNPK -10%),Percent Change in Rice_Production (TotalNPK +10%),Percent Change in Rice_Production (TotalNPK -10%),Maize_Production (TotalNPK +10%),Maize_Production (TotalNPK -10%),...,Maize_Production (TotalNPK +20%),Maize_Production (TotalNPK -20%),Normalized Maize_Production (TotalNPK +20%),Normalized Maize_Production (TotalNPK -20%),Percent Change in Maize_Production (TotalNPK +20%),Percent Change in Maize_Production (TotalNPK -20%),New Stability(NPK + 10%)_Rice_Maize,New Stability(NPK - 10%)_Rice_Maize,New Stability(NPK + 20%)_Rice_Maize,New Stability(NPK - 20%)_Rice_Maize
0,BENGALURU,0.97384,28742.13738,27510.79250,0.01476,0.01330,751.36663,714.89314,23294.31502,22015.48723,...,23933.72892,21376.07334,0.00907,0.00605,731.32091,642.48258,0.99048,0.99369,0.98888,0.99530
1,BENGALURU(R),0.98397,24181.87905,23779.67205,0.00937,0.00889,927.26759,910.18148,64995.11946,56134.32722,...,69425.51557,51703.93111,0.06287,0.04192,63.03576,21.41918,0.99202,0.99495,0.99055,0.99641
2,RAMANAGARA,0.86835,25369.08507,24751.02242,0.01077,0.01004,109.17781,104.08165,19827.86325,19179.29942,...,20152.14516,18855.01750,0.00460,0.00307,92.84349,80.43079,0.90663,0.92509,0.89740,0.93431
3,CHITRADURGA,0.56271,24382.18965,23943.56254,0.00960,0.00909,433.99452,424.38814,191263.30935,159444.66441,...,207172.63182,143535.34194,0.22577,0.15052,-23.82659,-47.22480,0.74786,0.79519,0.72420,0.81885
4,DAVANAGERE,0.07259,301361.83678,250563.27382,0.33716,0.27709,-38.43539,-48.81293,428835.88026,353822.22243,...,466342.70918,316315.39351,0.53227,0.35485,-21.43081,-46.70734,0.63799,0.70529,0.60434,0.73894
5,KOLAR,0.91809,23399.10248,23139.21849,0.00844,0.00813,2608.22945,2578.15029,19804.71100,19160.35667,...,20126.88817,18838.17950,0.00457,0.00305,1044.22332,970.95961,0.92421,0.93947,0.91658,0.94710
6,CHIKKABALLAPURA,0.88384,24732.89312,24230.50174,0.01002,0.00943,816.71212,798.09124,147835.18114,123912.55951,...,159796.49195,111951.24869,0.16975,0.11316,51.47879,6.12398,0.86648,0.89224,0.85361,0.90512
7,SHIVAMOGGA,0.63073,255553.14562,213083.43561,0.28299,0.23277,-33.61808,-44.64992,183652.21335,153217.40405,...,198869.61801,137999.99940,0.21596,0.14397,-12.21125,-39.08145,0.86893,0.89424,0.85628,0.90690
8,TUMAKURU,0.86588,36067.00998,33503.87008,0.02342,0.02039,38.49555,28.65321,60343.86239,52328.75326,...,64351.41695,48321.19869,0.05687,0.03792,-3.76351,-27.73644,0.87528,0.89944,0.86321,0.91152
9,CHIKKAMAGALURU,0.69899,215646.58440,180432.61279,0.23580,0.19415,95.09702,63.23868,147183.37828,123379.26626,...,159085.43429,111477.21025,0.16891,0.11260,159.29528,81.69806,0.88802,0.91000,0.87703,0.92099


This is the code used to get a different visualizations, previous codes we were used to observe how any change effected the calculations, this one is to get the best visualizations

In [85]:
import statsmodels.api as sm
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

def calc_and_vis_impact_stability_2D(G, existing_data, adjacency_file, factor, base_column, capability_vector, change_percentages, dim):
    min_value = float('inf')
    max_value = float('-inf')
    
    change_min_value = float('inf')
    change_max_value = float('-inf')
    
    initial_stability_column = calculate_and_return_initial_stability(df, capability_vector, G)
    existing_data = pd.concat([existing_data, initial_stability_column], axis=1)

    col_pass_plus = []
    col_pass_minus = []
    result_df = pd.DataFrame()
    
    for intervention in change_percentages:
        for cap_col in capability_vector:
            df['temp'] = df[factor]*df[cap_col]
            X = sm.add_constant(df['temp'])
            y = df[cap_col]
            model = sm.OLS(y, X).fit()
            m, c = model.params['temp'], model.params['const']

            base_column_increase = (1 + intervention/100) * df['temp']
            base_column_decrease = (1 - intervention/100) * df['temp']

            change_vector_increase = m * base_column_increase + c
            change_vector_decrease = m * base_column_decrease + c
            
            change_min_value = min(change_min_value, change_vector_increase.min(), change_vector_decrease.min())
            change_max_value = max(change_max_value, change_vector_increase.max(), change_vector_decrease.max())
            
    result_df = pd.DataFrame()
    res1_df = pd.DataFrame()
    
    for intervention in change_percentages:
        col_pass_plus = []
        col_pass_minus = []
        for cap_col in capability_vector:
            # Calculate Impact Score
            # filtered_df = df[(df[cap_col] != 0) & (~df[cap_col].isnull())]
            # filtered_df['temp'] = filtered_df[factor]*filtered_df[cap_col]
            df['temp'] = df[factor]*df[cap_col]
            X = sm.add_constant(df['temp'])
            y = df[cap_col]
            model = sm.OLS(y, X).fit()

            m, c = model.params['temp'], model.params['const']
            print(m,c,cap_col,intervention)
            
            base_column_increase = (1 + intervention/100) * df['temp']
            base_column_decrease = (1 - intervention/100) * df['temp']

            new_vector_increase = m * base_column_increase + c
            new_vector_decrease = m * base_column_decrease + c
            
            normalized_new_increase_vector = (new_vector_increase - change_min_value) / (change_max_value - change_min_value)
            normalized_new_decrease_vector = (new_vector_decrease - change_min_value) / (change_max_value - change_min_value)
            
            percent_change_inc = (new_vector_increase - (y) )*100/(y)
            percent_change_dec = (new_vector_decrease - (y) )*100/(y)

            result_df = pd.concat([result_df,
                pd.DataFrame({
                    f'{cap_col} ({base_column} +{intervention}%)': new_vector_increase,
                    f'{cap_col} ({base_column} -{intervention}%)': new_vector_decrease,
                    f'Normalized {cap_col} ({base_column} +{intervention}%)': normalized_new_increase_vector,
                    f'Normalized {cap_col} ({base_column} -{intervention}%)': normalized_new_decrease_vector,
                    f'Percent Change in {cap_col} ({base_column} +{intervention}%)': percent_change_inc,
                    f'Percent Change in {cap_col} ({base_column} -{intervention}%)': percent_change_dec,
                    
                })
            ], axis=1)

            columns_to_pass = f'{cap_col} ({base_column} +{intervention}%)'
            col_pass_plus.append(columns_to_pass)
            
            # Calculate Stability for the opposite intervention and add to DataFrame
            opposite_intervention = -intervention
            columns_to_pass_opposite = f'{cap_col} ({base_column} -{intervention}%)'
            col_pass_minus.append(columns_to_pass_opposite)
        
        for column in result_df.columns:
            existing_data[column] = result_df[column]

        # print(col_pass_minus)
        # print(col_pass_plus)
        
        stability_column, stability_column_name = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, col_pass_plus, intervention, dim)

        normalized_stability_vector = (stability_column - stability_column.min()) / (stability_column.max() - stability_column.min())

        capability_first_words = [col.split('_')[0] for col in capability_vector]
        stability_column_name_modified = stability_column_name + "_" + "_".join(capability_first_words)
        existing_data[stability_column_name_modified] = normalized_stability_vector
        
        stability_column_opposite, stability_column_name_opposite = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, col_pass_minus, opposite_intervention, dim)

        # Normalize stability values for the opposite intervention
        normalized_stability_vector_opposite = (stability_column_opposite - stability_column_opposite.min()) / (stability_column_opposite.max() - stability_column_opposite.min())    
        stability_column_name_opposite_modified = stability_column_name_opposite + "_" + "_".join(capability_first_words)
        existing_data[stability_column_name_opposite_modified] = normalized_stability_vector_opposite
        
        res1_df = pd.concat([res1_df,
            pd.DataFrame({
            stability_column_name_modified: stability_column,
            stability_column_name_opposite_modified: stability_column_opposite,
            })
        ], axis=1)
        
        normalized_data_1 = res1_df.copy()
        min_value = normalized_data_1.min().min()
        max_value = normalized_data_1.max().max()
        normalized_data_1 = (normalized_data_1 - min_value) / (max_value - min_value)
        
        # for col in normalized_data_1.columns:
        for column in normalized_data_1.columns:
            existing_data[column] = normalized_data_1[column]
        
        # Visualize with Plotly for +intervention
        fig_increase = go.Figure()
        color_palette = ['red', 'blue', 'green', 'orange', 'purple'] 

        for i, cap_col in enumerate(capability_vector):
            symbol = 'star' if i == 0 else 'triangle-up' 
            scatter_increase = go.Scatter(
                x=existing_data[stability_column_name_modified],
                y=existing_data[f"Percent Change in {cap_col} ({base_column} +{abs(intervention)}%)"],
                mode='markers',
                marker=dict(size=7, symbol=symbol, color=color_palette[i]),  
                name=f'{cap_col} +{intervention}%',
                text=existing_data['District'],
                hovertemplate='%{text}' + '<br>Percent Change in: %{y:.2f}' + '<br>Stability: %{x:.2f}'
            )
            fig_increase.add_trace(scatter_increase)
        
        # Add average lines
        # print("aefawefaw")
        avg_stability_increase = existing_data[stability_column_name_modified].mean()
        avg_impact_increase = existing_data[f"Percent Change in {capability_vector[0]} ({base_column} {'+' if intervention >= 0 else '-'}{abs(intervention)}%)"].mean()
        y_range_1 = max(abs(existing_data[f"Percent Change in {capability_vector[0]} ({base_column} +{abs(intervention)}%)"].min()), 
                    abs(existing_data[f"Percent Change in {capability_vector[0]} ({base_column} -{abs(intervention)}%)"].min()), 
                    abs(existing_data[f"Percent Change in {capability_vector[0]} ({base_column} +{abs(intervention)}%)"].max()), 
                    abs(existing_data[f"Percent Change in {capability_vector[0]} ({base_column} -{abs(intervention)}%)"].max()))
        y_axis_1 = max(y_range_1,1)


        fig_increase.add_shape(
            go.layout.Shape(
                type="line",
                x0=avg_stability_increase,
                x1=avg_stability_increase,
                y0=0,
                y1=y_axis_1,
                line=dict(color="black", dash="dash"),
                name="Average Stability"
            )
        )
        fig_increase.add_shape(
            go.layout.Shape(
                type="line",
                x0=0,
                x1=1,
                y0=avg_impact_increase,
                y1=avg_impact_increase,
                line=dict(color="black", dash="dash"),
                name="Average Impact"
            )
        )

        # Update layout for +intervention plot
        fig_increase.update_layout(
            title=f'Percent Change vs Stability (+{intervention}%)',
            xaxis=dict(title=f'Stability ({base_column} +{intervention}%)'),
            yaxis=dict(title='Percent Change'),
            hovermode='closest'
        )

        # Visualize with Plotly for -intervention
        fig_decrease = go.Figure()

        for i, cap_col in enumerate(capability_vector):
            # Scatter plot for -intervention
            symbol = 'star' if i == 0 else 'triangle-up'
            
            scatter_decrease = go.Scatter(
                x=existing_data[stability_column_name_opposite_modified],
                y=existing_data[f"Percent Change in {cap_col} ({base_column} {'+' if intervention <= 0 else '-'}{abs(intervention)}%)"],
                mode='markers',
                marker=dict(size=7,symbol=symbol, color=color_palette[i]),
                name=f'{cap_col} -{intervention}%',
                text=existing_data['District'],
                hovertemplate='%{text}' + '<br>Percent Change: %{y:.2f}' + '<br>Stability: %{x:.2f}'
            )
            
            fig_decrease.add_trace(scatter_decrease)

        # Add average lines for -intervention
        avg_stability_decrease = existing_data[stability_column_name_opposite_modified].mean()
        avg_impact_decrease = existing_data[f"Percent Change in {capability_vector[0]} ({base_column} {'+' if intervention <= 0 else '-'}{abs(intervention)}%)"].mean()

        fig_decrease.add_shape(
            go.layout.Shape(
                type="line",
                x0=avg_stability_decrease,
                x1=avg_stability_decrease,
                y0=0,
                y1=y_axis_1,
                line=dict(color="black", dash="dash"),
                name="Average Stability"
            )
        )

        fig_decrease.add_shape(
            go.layout.Shape(
                type="line",
                x0=0,
                x1=1,
                y0=avg_impact_decrease,
                y1=avg_impact_decrease,
                line=dict(color="black", dash="dash"),
                name="Average Impact"
            )
        )

        # Update layout for -intervention plot
        fig_decrease.update_layout(
            title=f'Percent Change vs Stability (-{intervention}%)',
            xaxis=dict(title=f'Stability ({base_column} -{intervention}%)'),
            yaxis=dict(title='Percent Change'),
            hovermode='closest'
        )


        fig_increase.show()
        fig_decrease.show()

    return existing_data, result_df

base_column = "TotalNPK"
capability_vector = ["Rice_Production", "Maize_Production"]
factor = "FCR"
# change_percentage = 20
change_percentages = [10,20]
dim = len(capability_vector)

new_data_2,result_df = calc_and_vis_impact_stability_2D(G, existing_data.copy(), adjacency_file, factor, base_column, capability_vector, change_percentages, dim)
new_data_2.head()

11.9821894393767 21969.740534991743 Rice_Production 10
14.592486866377046 16260.762174034226 Maize_Production 10


11.9821894393767 21969.740534991743 Rice_Production 20
14.592486866377046 16260.762174034226 Maize_Production 20


,District,Initial Stability,Rice_Production (TotalNPK +10%),Rice_Production (TotalNPK -10%),Normalized Rice_Production (TotalNPK +10%),Normalized Rice_Production (TotalNPK -10%),Percent Change in Rice_Production (TotalNPK +10%),Percent Change in Rice_Production (TotalNPK -10%),Maize_Production (TotalNPK +10%),Maize_Production (TotalNPK -10%),...,Percent Change in Rice_Production (TotalNPK +20%),Percent Change in Rice_Production (TotalNPK -20%),Maize_Production (TotalNPK +20%),Maize_Production (TotalNPK -20%),Normalized Maize_Production (TotalNPK +20%),Normalized Maize_Production (TotalNPK -20%),Percent Change in Maize_Production (TotalNPK +20%),Percent Change in Maize_Production (TotalNPK -20%),New Stability(NPK + 20%)_Rice_Maize,New Stability(NPK - 20%)_Rice_Maize
0,BENGALURU,0.97384,28742.13738,27510.79250,0.01476,0.01330,751.36663,714.89314,23294.31502,22015.48723,...,769.60337,696.65640,23933.72892,21376.07334,0.00907,0.00605,731.32091,642.48258,0.98888,0.99530
1,BENGALURU(R),0.98397,24181.87905,23779.67205,0.00937,0.00889,927.26759,910.18148,64995.11946,56134.32722,...,935.81064,901.63843,69425.51557,51703.93111,0.06287,0.04192,63.03576,21.41918,0.99055,0.99641
2,RAMANAGARA,0.86835,25369.08507,24751.02242,0.01077,0.01004,109.17781,104.08165,19827.86325,19179.29942,...,111.72589,101.53357,20152.14516,18855.01750,0.00460,0.00307,92.84349,80.43079,0.89740,0.93431
3,CHITRADURGA,0.56271,24382.18965,23943.56254,0.00960,0.00909,433.99452,424.38814,191263.30935,159444.66441,...,438.79770,419.58495,207172.63182,143535.34194,0.22577,0.15052,-23.82659,-47.22480,0.72420,0.81885
4,DAVANAGERE,0.07259,301361.83678,250563.27382,0.33716,0.27709,-38.43539,-48.81293,428835.88026,353822.22243,...,-33.24662,-54.00170,466342.70918,316315.39351,0.53227,0.35485,-21.43081,-46.70734,0.60434,0.73894


In [86]:
# new_data_2

In [87]:
df

,District,TotalNPK,Rice_Production,Jowar_Production,Maize_Production,Jowar_Yield,TotalCerealsandMinorMillets_Production,TotalOilSeeds_Production,TotalFoodGrains,TotalGourdVarietyVegetables,...,temp,Abbreviation,Relative Change in Rice_Production (TotalNPK -10%),Relative Change in Maize_Production (TotalNPK -10%),Relative Change in Rice_Production (TotalNPK +10%),Relative Change in Maize_Production (TotalNPK +10%),Relative Change in Rice_Production (TotalNPK +20%),Relative Change in Maize_Production (TotalNPK +20%),Relative Change in Rice_Production (TotalNPK -20%),Relative Change in Maize_Production (TotalNPK -20%)
0,BENGALURU,23310.00000,3376.00000,851,2879,1539.00000,69153.00000,162.00000,72752.00000,11088.00000,...,438.18021,BLR,7.14893,6.64692,7.51367,7.09111,7.69603,7.31321,6.96656,6.42483
1,BENGALURU(R),19259.00000,2354.00000,0,42583,0.00000,125232.00000,479.00000,131795.00000,12614.00000,...,3036.08025,BLR(R),9.10181,0.31823,9.27268,0.52632,9.35811,0.63036,9.01638,0.21419
2,RAMANAGARA,7472.00000,12128.00000,0,10450,0.00000,159008.00000,6414.00000,175873.00000,10071.00000,...,222.22526,RGA,1.04082,0.83534,1.09178,0.89740,1.11726,0.92843,1.01534,0.80431
3,CHITRADURGA,35884.00000,4566.00000,13301,271975,984.00000,388849.00000,78257.00000,421955.00000,6113.00000,...,10902.40657,CDA,4.24388,-0.41375,4.33995,-0.29676,4.38798,-0.23827,4.19585,-0.47225
4,DAVANAGERE,99644.00000,489505.00000,23072,593544,2167.00000,1126753.00000,21327.00000,1141132.00000,11822.00000,...,25702.83548,DVG,-0.48813,-0.40388,-0.38435,-0.27750,-0.33247,-0.21431,-0.54002,-0.46707
5,KOLAR,22190.00000,864.00000,0,1759,0.00000,69089.00000,5994.00000,87015.00000,14692.00000,...,220.78291,KLR,25.78150,9.89276,26.08229,10.25907,26.23269,10.44223,25.63111,9.70960
6,CHIKKABALLAPURA,28357.00000,2698.00000,0,105491,0.00000,165723.00000,7341.00000,171962.00000,19918.00000,...,8196.89675,CKA,7.98091,0.17463,8.16712,0.40140,8.26023,0.51479,7.88781,0.06124
7,SHIVAMOGGA,56609.00000,384974.00000,756,226532,2242.00000,613331.00000,2146.00000,613998.00000,238.00000,...,10428.24626,SMG,-0.44650,-0.32364,-0.33618,-0.18929,-0.28102,-0.12211,-0.50166,-0.39081
8,TUMAKURU,38148.00000,26042.00000,2652,66868,1346.00000,431562.00000,42331.00000,450563.00000,4382.00000,...,2746.31363,TKR,0.28653,-0.21743,0.38496,-0.09757,0.43417,-0.03764,0.23732,-0.27736
9,CHIKKAMAGALURU,67863.00000,110533.00000,9080,61353,897.00000,235695.00000,8612.00000,253087.00000,13083.00000,...,8156.29037,CMG,0.63239,1.01097,0.95097,1.39896,1.11026,1.59295,0.47310,0.81698


Similar to the previous code, but this calculates the relative change instead of the percent change

In [88]:
import statsmodels.api as sm
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

def calc_and_vis_impact_stability_2D(G, existing_data, adjacency_file, factor, base_column, capability_vector, change_percentages, dim):
    min_value = float('inf')
    max_value = float('-inf')
    
    change_min_value = float('inf')
    change_max_value = float('-inf')
    
    initial_stability_column = calculate_and_return_initial_stability(df, capability_vector, G)
    existing_data = pd.concat([existing_data, initial_stability_column], axis=1)

    col_pass_plus = []
    col_pass_minus = []
    result_df = pd.DataFrame()
    
    for intervention in change_percentages:
        for cap_col in capability_vector:
            df['temp'] = df[factor]*df[cap_col]
            X = sm.add_constant(df['temp'])
            y = df[cap_col]
            model = sm.OLS(y, X).fit()
            m, c = model.params['temp'], model.params['const']

            base_column_increase = (1 + intervention/100) * df['temp']
            base_column_decrease = (1 - intervention/100) * df['temp']

            change_vector_increase = m * base_column_increase + c
            change_vector_decrease = m * base_column_decrease + c
            
            change_min_value = min(change_min_value, change_vector_increase.min(), change_vector_decrease.min())
            change_max_value = max(change_max_value, change_vector_increase.max(), change_vector_decrease.max())
            
    result_df = pd.DataFrame()
    res1_df = pd.DataFrame()
    
    for intervention in change_percentages:
        col_pass_plus = []
        col_pass_minus = []
        for cap_col in capability_vector:
            # Calculate Impact Score
            # filtered_df = df[(df[cap_col] != 0) & (~df[cap_col].isnull())]
            # filtered_df['temp'] = filtered_df[factor]*filtered_df[cap_col]
            df['temp'] = df[factor]*df[cap_col]
            X = sm.add_constant(df['temp'])
            y = df[cap_col]
            model = sm.OLS(y, X).fit()

            m, c = model.params['temp'], model.params['const']
            print(m,c,cap_col,intervention)
            
            base_column_increase = (1 + intervention/100) * df['temp']
            base_column_decrease = (1 - intervention/100) * df['temp']

            new_vector_increase = m * base_column_increase + c
            new_vector_decrease = m * base_column_decrease + c
            
            normalized_new_increase_vector = (new_vector_increase - change_min_value) / (change_max_value - change_min_value)
            normalized_new_decrease_vector = (new_vector_decrease - change_min_value) / (change_max_value - change_min_value)
            
            percent_change_inc = (new_vector_increase - (y) )/(y)
            percent_change_dec = (new_vector_decrease - (y) )/(y)

            result_df = pd.concat([result_df,
                pd.DataFrame({
                    f'{cap_col} ({base_column} +{intervention}%)': new_vector_increase,
                    f'{cap_col} ({base_column} -{intervention}%)': new_vector_decrease,
                    f'Normalized {cap_col} ({base_column} +{intervention}%)': normalized_new_increase_vector,
                    f'Normalized {cap_col} ({base_column} -{intervention}%)': normalized_new_decrease_vector,
                    f'Percent Change in {cap_col} ({base_column} +{intervention}%)': percent_change_inc,
                    f'Percent Change in {cap_col} ({base_column} -{intervention}%)': percent_change_dec,
                    
                })
            ], axis=1)

            columns_to_pass = f'{cap_col} ({base_column} +{intervention}%)'
            col_pass_plus.append(columns_to_pass)
            
            # Calculate Stability for the opposite intervention and add to DataFrame
            opposite_intervention = -intervention
            columns_to_pass_opposite = f'{cap_col} ({base_column} -{intervention}%)'
            col_pass_minus.append(columns_to_pass_opposite)
        
        for column in result_df.columns:
            existing_data[column] = result_df[column]

        # print(col_pass_minus)
        # print(col_pass_plus)
        
        stability_column, stability_column_name = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, col_pass_plus, intervention, dim)

        normalized_stability_vector = (stability_column - stability_column.min()) / (stability_column.max() - stability_column.min())

        capability_first_words = [col.split('_')[0] for col in capability_vector]
        stability_column_name_modified = stability_column_name + "_" + "_".join(capability_first_words)
        existing_data[stability_column_name_modified] = normalized_stability_vector
        
        stability_column_opposite, stability_column_name_opposite = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, col_pass_minus, opposite_intervention, dim)

        # Normalize stability values for the opposite intervention
        normalized_stability_vector_opposite = (stability_column_opposite - stability_column_opposite.min()) / (stability_column_opposite.max() - stability_column_opposite.min())    
        stability_column_name_opposite_modified = stability_column_name_opposite + "_" + "_".join(capability_first_words)
        existing_data[stability_column_name_opposite_modified] = normalized_stability_vector_opposite
        
        res1_df = pd.concat([res1_df,
            pd.DataFrame({
            stability_column_name_modified: stability_column,
            stability_column_name_opposite_modified: stability_column_opposite,
            })
        ], axis=1)
        
        normalized_data_1 = res1_df.copy()
        min_value = normalized_data_1.min().min()
        max_value = normalized_data_1.max().max()
        normalized_data_1 = (normalized_data_1 - min_value) / (max_value - min_value)
        
        # for col in normalized_data_1.columns:
        for column in normalized_data_1.columns:
            existing_data[column] = normalized_data_1[column]
        
        # Visualize with Plotly for +intervention
        fig_increase = go.Figure()
        color_palette = ['red', 'blue', 'green', 'orange', 'purple'] 

        for i, cap_col in enumerate(capability_vector):
            symbol = 'star' if i == 0 else 'triangle-up' 
            scatter_increase = go.Scatter(
                x=existing_data[stability_column_name_modified],
                y=existing_data[f"Percent Change in {cap_col} ({base_column} +{abs(intervention)}%)"],
                mode='markers',
                marker=dict(size=7, symbol=symbol, color=color_palette[i]),  
                name=f'{cap_col} +{intervention}%',
                text=existing_data['District'],
                hovertemplate='%{text}' + '<br>Percent Change in: %{y:.2f}' + '<br>Stability: %{x:.2f}'
            )
            fig_increase.add_trace(scatter_increase)
        
        # Add average lines
        # print("aefawefaw")
        avg_stability_increase = existing_data[stability_column_name_modified].mean()
        avg_impact_increase = existing_data[f"Percent Change in {capability_vector[0]} ({base_column} {'+' if intervention >= 0 else '-'}{abs(intervention)}%)"].mean()
        y_range_1 = max(abs(existing_data[f"Percent Change in {capability_vector[0]} ({base_column} +{abs(intervention)}%)"].min()), 
                    abs(existing_data[f"Percent Change in {capability_vector[0]} ({base_column} -{abs(intervention)}%)"].min()), 
                    abs(existing_data[f"Percent Change in {capability_vector[0]} ({base_column} +{abs(intervention)}%)"].max()), 
                    abs(existing_data[f"Percent Change in {capability_vector[0]} ({base_column} -{abs(intervention)}%)"].max()))
        y_axis_1 = max(y_range_1,1)


        fig_increase.add_shape(
            go.layout.Shape(
                type="line",
                x0=avg_stability_increase,
                x1=avg_stability_increase,
                y0=0,
                y1=y_axis_1,
                line=dict(color="black", dash="dash"),
                name="Average Stability"
            )
        )
        fig_increase.add_shape(
            go.layout.Shape(
                type="line",
                x0=0,
                x1=1,
                y0=avg_impact_increase,
                y1=avg_impact_increase,
                line=dict(color="black", dash="dash"),
                name="Average Impact"
            )
        )

        # Update layout for +intervention plot
        fig_increase.update_layout(
            title=f'Relative Change vs Stability (+{intervention}%)',
            xaxis=dict(title=f'Stability ({base_column} +{intervention}%)'),
            yaxis=dict(title='Relative Change'),
            hovermode='closest'
        )

        # Visualize with Plotly for -intervention
        fig_decrease = go.Figure()

        for i, cap_col in enumerate(capability_vector):
            # Scatter plot for -intervention
            symbol = 'star' if i == 0 else 'triangle-up'
            
            scatter_decrease = go.Scatter(
                x=existing_data[stability_column_name_opposite_modified],
                y=existing_data[f"Percent Change in {cap_col} ({base_column} {'+' if intervention <= 0 else '-'}{abs(intervention)}%)"],
                mode='markers',
                marker=dict(size=7,symbol=symbol, color=color_palette[i]),
                name=f'{cap_col} -{intervention}%',
                text=existing_data['District'],
                hovertemplate='%{text}' + '<br>Relative Change: %{y:.2f}' + '<br>Stability: %{x:.2f}'
            )
            
            fig_decrease.add_trace(scatter_decrease)

        # Add average lines for -intervention
        avg_stability_decrease = existing_data[stability_column_name_opposite_modified].mean()
        avg_impact_decrease = existing_data[f"Percent Change in {capability_vector[0]} ({base_column} {'+' if intervention <= 0 else '-'}{abs(intervention)}%)"].mean()

        fig_decrease.add_shape(
            go.layout.Shape(
                type="line",
                x0=avg_stability_decrease,
                x1=avg_stability_decrease,
                y0=0,
                y1=y_axis_1,
                line=dict(color="black", dash="dash"),
                name="Average Stability"
            )
        )

        fig_decrease.add_shape(
            go.layout.Shape(
                type="line",
                x0=0,
                x1=1,
                y0=avg_impact_decrease,
                y1=avg_impact_decrease,
                line=dict(color="black", dash="dash"),
                name="Average Impact"
            )
        )

        # Update layout for -intervention plot
        fig_decrease.update_layout(
            title=f'Relative Change vs Stability (-{intervention}%)',
            xaxis=dict(title=f'Stability ({base_column} -{intervention}%)'),
            yaxis=dict(title='Relative Change'),
            hovermode='closest'
        )


        fig_increase.show()
        fig_decrease.show()

    return existing_data, result_df

base_column = "TotalNPK"
capability_vector = ["Rice_Production", "Maize_Production"]
factor = "FCR"
# change_percentage = 20
change_percentages = [10,20]
dim = len(capability_vector)

new_data_2,result_df = calc_and_vis_impact_stability_2D(G, existing_data.copy(), adjacency_file, factor, base_column, capability_vector, change_percentages, dim)
new_data_2.head()

11.9821894393767 21969.740534991743 Rice_Production 10
14.592486866377046 16260.762174034226 Maize_Production 10


11.9821894393767 21969.740534991743 Rice_Production 20
14.592486866377046 16260.762174034226 Maize_Production 20


,District,Initial Stability,Rice_Production (TotalNPK +10%),Rice_Production (TotalNPK -10%),Normalized Rice_Production (TotalNPK +10%),Normalized Rice_Production (TotalNPK -10%),Percent Change in Rice_Production (TotalNPK +10%),Percent Change in Rice_Production (TotalNPK -10%),Maize_Production (TotalNPK +10%),Maize_Production (TotalNPK -10%),...,Percent Change in Rice_Production (TotalNPK +20%),Percent Change in Rice_Production (TotalNPK -20%),Maize_Production (TotalNPK +20%),Maize_Production (TotalNPK -20%),Normalized Maize_Production (TotalNPK +20%),Normalized Maize_Production (TotalNPK -20%),Percent Change in Maize_Production (TotalNPK +20%),Percent Change in Maize_Production (TotalNPK -20%),New Stability(NPK + 20%)_Rice_Maize,New Stability(NPK - 20%)_Rice_Maize
0,BENGALURU,0.97384,28742.13738,27510.79250,0.01476,0.01330,7.51367,7.14893,23294.31502,22015.48723,...,7.69603,6.96656,23933.72892,21376.07334,0.00907,0.00605,7.31321,6.42483,0.98888,0.99530
1,BENGALURU(R),0.98397,24181.87905,23779.67205,0.00937,0.00889,9.27268,9.10181,64995.11946,56134.32722,...,9.35811,9.01638,69425.51557,51703.93111,0.06287,0.04192,0.63036,0.21419,0.99055,0.99641
2,RAMANAGARA,0.86835,25369.08507,24751.02242,0.01077,0.01004,1.09178,1.04082,19827.86325,19179.29942,...,1.11726,1.01534,20152.14516,18855.01750,0.00460,0.00307,0.92843,0.80431,0.89740,0.93431
3,CHITRADURGA,0.56271,24382.18965,23943.56254,0.00960,0.00909,4.33995,4.24388,191263.30935,159444.66441,...,4.38798,4.19585,207172.63182,143535.34194,0.22577,0.15052,-0.23827,-0.47225,0.72420,0.81885
4,DAVANAGERE,0.07259,301361.83678,250563.27382,0.33716,0.27709,-0.38435,-0.48813,428835.88026,353822.22243,...,-0.33247,-0.54002,466342.70918,316315.39351,0.53227,0.35485,-0.21431,-0.46707,0.60434,0.73894


# For Tableau Dataset:

This is code is to get the data in a format that is easier to use in tableau for the kind of visualizations we wanted to make in tableau.
All the new changed values are in one column in this data, and there is a column that shows the perturbation done to get it, this makes it easier to choose the perturbation we want to use in tableau than change the column name, and have too many of them

In [89]:
import statsmodels.api as sm
import pandas as pd
import plotly.graph_objects as go

def calc_and_vis_impact_stability_2D(G, existing_data, adjacency_file, factor , base_column, capability_vector, change_percentages, dim):
    # Calculate min and max values for changes made by each column in capability_vector
    min_value = float('inf')
    max_value = float('-inf')
    
    change_min_value = float('inf')
    change_max_value = float('-inf')
    
    initial_stability_column = calculate_and_return_initial_stability(df, capability_vector, G)
    existing_data = pd.concat([existing_data, initial_stability_column], axis=1)

    col_pass_plus = []
    col_pass_minus = []
    result_df = pd.DataFrame()

    # Create fixed columns
    fixed_columns = {
        'District': [],
        'Initial_Stability': [],
        'Base_Column': [],
        'Intervention_Percentage': [],
        'Capability_Vector': [],
        'New_Values': [],
        'Normalized_New_Values': [],
        'Percent_Change': [],
        'Relative_Change':[]
    }
    
    for intervention in change_percentages:
        for cap_col in capability_vector:
            df['temp'] = df[factor]*df[cap_col]
            X = sm.add_constant(df['temp'])
            y = df[cap_col]
            model = sm.OLS(y, X).fit()
            print(cap_col)            
            m, c = model.params['temp'], model.params['const']
            print(m,c)
            
            base_column_increase = (1 + intervention/100) * df['temp']
            base_column_decrease = (1 - intervention/100) * df['temp']

            change_vector_increase = m * base_column_increase + c
            change_vector_decrease = m * base_column_decrease + c
            
            change_min_value = min(change_min_value, change_vector_increase.min(), change_vector_decrease.min())
            change_max_value = max(change_max_value, change_vector_increase.max(), change_vector_decrease.max())

    for intervention in change_percentages:
        for cap_col in capability_vector:
            df['temp'] = df[factor]*df[cap_col]
            X = sm.add_constant(df['temp'])
            y = df[cap_col]
            model = sm.OLS(y, X).fit()

            m, c = model.params['temp'], model.params['const']
            base_column_increase = (1 + intervention/100) * df['temp']
            base_column_decrease = (1 - intervention/100) * df['temp']

            new_vector_increase = m * base_column_increase + c
            new_vector_decrease = m * base_column_decrease + c
            
            normalized_new_increase_vector = (new_vector_increase - change_min_value) / (change_max_value - change_min_value)
            normalized_new_decrease_vector = (new_vector_decrease - change_min_value) / (change_max_value - change_min_value)
            
            percent_change_inc = (new_vector_increase - (y) )*100/(y)
            percent_change_dec = (new_vector_decrease - (y) )*100/(y)
            
            relative_change_inc = (new_vector_increase - (y) )/(y)
            relative_change_dec = (new_vector_decrease - (y) )/(y)

            fixed_columns['District'].extend(existing_data['District'].tolist())
            fixed_columns['Initial_Stability'].extend(existing_data['Initial Stability'].tolist())
            fixed_columns['Base_Column'].extend([base_column] * len(existing_data))
            fixed_columns['Intervention_Percentage'].extend([intervention] * len(existing_data))
            fixed_columns['Capability_Vector'].extend([cap_col] * len(existing_data))
            fixed_columns['New_Values'].extend(new_vector_increase.tolist())
            fixed_columns['Normalized_New_Values'].extend(normalized_new_increase_vector.tolist())
            fixed_columns['Percent_Change'].extend(percent_change_inc.tolist())
            fixed_columns['Relative_Change'].extend(relative_change_inc.tolist())

            fixed_columns['District'].extend(existing_data['District'].tolist())
            fixed_columns['Initial_Stability'].extend(existing_data['Initial Stability'].tolist())
            fixed_columns['Base_Column'].extend([base_column] * len(existing_data))
            fixed_columns['Intervention_Percentage'].extend([-intervention] * len(existing_data))
            fixed_columns['Capability_Vector'].extend([cap_col] * len(existing_data))
            fixed_columns['New_Values'].extend(new_vector_decrease.tolist())
            fixed_columns['Normalized_New_Values'].extend(normalized_new_decrease_vector.tolist())
            fixed_columns['Percent_Change'].extend(percent_change_dec.tolist())
            fixed_columns['Relative_Change'].extend(relative_change_dec.tolist())
            
    # Create DataFrame from fixed columns
    result_df = pd.DataFrame(fixed_columns)

    return result_df

# Usage
base_column = "TotalNPK"
capability_vector = ["Rice_Production", "Maize_Production"]
adjacency_file = "Karnataka_District_Adjacency_File - Copy.xlsx"
change_percentages = [10, 20]
dim = len(capability_vector)

result_df_2 = calc_and_vis_impact_stability_2D(G, existing_data.copy(), adjacency_file, factor, base_column, capability_vector, change_percentages, dim)
result_df_2.head()


Rice_Production
11.9821894393767 21969.740534991743
Maize_Production
14.592486866377046 16260.762174034226
Rice_Production
11.9821894393767 21969.740534991743
Maize_Production
14.592486866377046 16260.762174034226


,District,Initial_Stability,Base_Column,Intervention_Percentage,Capability_Vector,New_Values,Normalized_New_Values,Percent_Change,Relative_Change
0,BENGALURU,0.97384,TotalNPK,10,Rice_Production,28742.13738,0.01476,751.36663,7.51367
1,BENGALURU(R),0.98397,TotalNPK,10,Rice_Production,24181.87905,0.00937,927.26759,9.27268
2,RAMANAGARA,0.86835,TotalNPK,10,Rice_Production,25369.08507,0.01077,109.17781,1.09178
3,CHITRADURGA,0.56271,TotalNPK,10,Rice_Production,24382.18965,0.00960,433.99452,4.33995
4,DAVANAGERE,0.07259,TotalNPK,10,Rice_Production,301361.83678,0.33716,-38.43539,-0.38435


In [90]:
result_df_2

,District,Initial_Stability,Base_Column,Intervention_Percentage,Capability_Vector,New_Values,Normalized_New_Values,Percent_Change,Relative_Change
0,BENGALURU,0.97384,TotalNPK,10,Rice_Production,28742.13738,0.01476,751.36663,7.51367
1,BENGALURU(R),0.98397,TotalNPK,10,Rice_Production,24181.87905,0.00937,927.26759,9.27268
2,RAMANAGARA,0.86835,TotalNPK,10,Rice_Production,25369.08507,0.01077,109.17781,1.09178
3,CHITRADURGA,0.56271,TotalNPK,10,Rice_Production,24382.18965,0.00960,433.99452,4.33995
4,DAVANAGERE,0.07259,TotalNPK,10,Rice_Production,301361.83678,0.33716,-38.43539,-0.38435
...,...,...,...,...,...,...,...,...,...
243,KALABURAGI,0.82659,TotalNPK,-20,Maize_Production,24550.48096,0.00980,67.91246,0.67912
244,YADGIRI,0.82384,TotalNPK,-20,Maize_Production,20563.99864,0.00509,669.32281,6.69323
245,RAICHUR,0.22877,TotalNPK,-20,Maize_Production,16979.26727,0.00085,2248.44637,22.48446
246,KOPPAL,0.82822,TotalNPK,-20,Maize_Production,144646.05003,0.15183,-23.97134,-0.23971


In [91]:
result_df_2 = pd.read_excel('Rice_Maize_10_20_31_FCR.xlsx')

In [92]:
result_df_2.to_excel('Rice_Maize_10_20_31_FCR.xlsx',index = False)

In [93]:
Stability_column_values = pd.concat([
                               new_data_1['New Stability(NPK + 10%)_Rice_Maize'], 
                               new_data_1['New Stability(NPK - 10%)_Rice_Maize'], 
                               new_data_1['New Stability(NPK + 10%)_Rice_Maize'], 
                               new_data_1['New Stability(NPK - 10%)_Rice_Maize'], 
                               new_data_1['New Stability(NPK + 20%)_Rice_Maize'], 
                               new_data_1['New Stability(NPK - 20%)_Rice_Maize'],
                               new_data_1['New Stability(NPK + 20%)_Rice_Maize'], 
                               new_data_1['New Stability(NPK - 20%)_Rice_Maize']], axis=0)

Stability_column_values.reset_index(drop=True, inplace=True)

result_df_2['Stability'] = Stability_column_values

In [94]:
result_df_2

,District,Initial_Stability,Base_Column,Intervention_Percentage,Capability_Vector,New_Values,Normalized_New_Values,Percent_Change,Relative_Change,Stability,Abbreviation,Legend
0,BENGALURU,0.97384,TotalNPK,10,Rice_Production,28742.13738,0.01476,751.36663,7.51367,0.99048,BLR,BENGALURU (BLR)
1,BENGALURU(R),0.98397,TotalNPK,10,Rice_Production,24181.87905,0.00937,927.26759,9.27268,0.99202,BLR(R),BENGALURU(R) (BLR(R))
2,RAMANAGARA,0.86835,TotalNPK,10,Rice_Production,25369.08507,0.01077,109.17781,1.09178,0.90663,RGA,RAMANAGARA (RGA)
3,CHITRADURGA,0.56271,TotalNPK,10,Rice_Production,24382.18965,0.00960,433.99452,4.33995,0.74786,CDA,CHITRADURGA (CDA)
4,DAVANAGERE,0.07259,TotalNPK,10,Rice_Production,301361.83678,0.33716,-38.43539,-0.38435,0.63799,DVG,DAVANAGERE (DVG)
...,...,...,...,...,...,...,...,...,...,...,...,...
243,KALABURAGI,0.82659,TotalNPK,-20,Maize_Production,24550.48096,0.00980,67.91246,0.67912,0.86293,KLB,KALABURAGI (KLB)
244,YADGIRI,0.82384,TotalNPK,-20,Maize_Production,20563.99864,0.00509,669.32281,6.69323,0.77111,YDR,YADGIRI (YDR)
245,RAICHUR,0.22877,TotalNPK,-20,Maize_Production,16979.26727,0.00085,2248.44637,22.48446,0.56811,RCR,RAICHUR (RCR)
246,KOPPAL,0.82822,TotalNPK,-20,Maize_Production,144646.05003,0.15183,-23.97134,-0.23971,0.97333,KPL,KOPPAL (KPL)


In [95]:
result_df

,Rice_Production (TotalNPK +10%),Rice_Production (TotalNPK -10%),Normalized Rice_Production (TotalNPK +10%),Normalized Rice_Production (TotalNPK -10%),Percent Change in Rice_Production (TotalNPK +10%),Percent Change in Rice_Production (TotalNPK -10%),Maize_Production (TotalNPK +10%),Maize_Production (TotalNPK -10%),Normalized Maize_Production (TotalNPK +10%),Normalized Maize_Production (TotalNPK -10%),...,Normalized Rice_Production (TotalNPK +20%),Normalized Rice_Production (TotalNPK -20%),Percent Change in Rice_Production (TotalNPK +20%),Percent Change in Rice_Production (TotalNPK -20%),Maize_Production (TotalNPK +20%),Maize_Production (TotalNPK -20%),Normalized Maize_Production (TotalNPK +20%),Normalized Maize_Production (TotalNPK -20%),Percent Change in Maize_Production (TotalNPK +20%),Percent Change in Maize_Production (TotalNPK -20%)
0,28742.13738,27510.79250,0.01476,0.01330,7.51367,7.14893,23294.31502,22015.48723,0.00832,0.00681,...,0.01549,0.01258,7.69603,6.96656,23933.72892,21376.07334,0.00907,0.00605,7.31321,6.42483
1,24181.87905,23779.67205,0.00937,0.00889,9.27268,9.10181,64995.11946,56134.32722,0.05763,0.04716,...,0.00961,0.00865,9.35811,9.01638,69425.51557,51703.93111,0.06287,0.04192,0.63036,0.21419
2,25369.08507,24751.02242,0.01077,0.01004,1.09178,1.04082,19827.86325,19179.29942,0.00422,0.00345,...,0.01114,0.00968,1.11726,1.01534,20152.14516,18855.01750,0.00460,0.00307,0.92843,0.80431
3,24382.18965,23943.56254,0.00960,0.00909,4.33995,4.24388,191263.30935,159444.66441,0.20696,0.16933,...,0.00986,0.00883,4.38798,4.19585,207172.63182,143535.34194,0.22577,0.15052,-0.23827,-0.47225
4,301361.83678,250563.27382,0.33716,0.27709,-0.38435,-0.48813,428835.88026,353822.22243,0.48792,0.39920,...,0.36720,0.24705,-0.33247,-0.54002,466342.70918,316315.39351,0.53227,0.35485,-0.21431,-0.46707
5,23399.10248,23139.21849,0.00844,0.00813,26.08229,25.78150,19804.71100,19160.35667,0.00419,0.00343,...,0.00860,0.00798,26.23269,25.63111,20126.88817,18838.17950,0.00457,0.00305,10.44223,9.70960
6,24732.89312,24230.50174,0.01002,0.00943,8.16712,7.98091,147835.18114,123912.55951,0.15560,0.12731,...,0.01032,0.00913,8.26023,7.88781,159796.49195,111951.24869,0.16975,0.11316,0.51479,0.06124
7,255553.14562,213083.43561,0.28299,0.23277,-0.33618,-0.44650,183652.21335,153217.40405,0.19796,0.16197,...,0.30810,0.20765,-0.28102,-0.50166,198869.61801,137999.99940,0.21596,0.14397,-0.12211,-0.39081
8,36067.00998,33503.87008,0.02342,0.02039,0.38496,0.28653,60343.86239,52328.75326,0.05213,0.04265,...,0.02494,0.01888,0.43417,0.23732,64351.41695,48321.19869,0.05687,0.03792,-0.03764,-0.27736
9,215646.58440,180432.61279,0.23580,0.19415,0.95097,0.63239,147183.37828,123379.26626,0.15483,0.12668,...,0.25662,0.17333,1.11026,0.47310,159085.43429,111477.21025,0.16891,0.11260,1.59295,0.81698


In [96]:
print(result_df['Rice_Production (TotalNPK +20%)'].min(), result_df['Rice_Production (TotalNPK +20%)'].max())

22081.13359770978 624607.8196873822


In [97]:
print(result_df['Maize_Production (TotalNPK +20%)'].min(), result_df['Maize_Production (TotalNPK +20%)'].max())

16260.762174034226 861845.4428063503
